In [39]:
import pandas as pd

verb_prefixes = pd.read_pickle("verb_prefixes.pkl")

In [40]:
verb_prefixes

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
5,11,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija,
16,6119,abonovati,NaN,v.tr. ipf.,"subscribe, engage",F,verb,?,?,abonovati,abonovati,abon,abonovati,,
28,19625,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,
29,24056,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,
43,24070,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,19473,žužati,NaN,v.intr. ipf.,buzz,NaN,verb,?,?,žužati,žužati,žuž,žužati,,
17913,-37005,råzpečętati,NaN,v.tr. pf.,unseal,NaN,verb,råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati,,
17914,-37006,råzpečętyvati,NaN,v.tr. ipf.,unseal,NaN,verb,råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?,,
17917,-30731,smogti,NaN,v.tr. pf.,"manage to, be able to, cope, make it",NaN,verb,?,?,mogti,smogti,smogti,mogti,,


In [ ]:
YI = "⒥"

In [572]:
AO = 'ꜵ'
YI = "յ"
NASAL = "˛"

In [307]:
import regex

AO_RE = regex.compile(f"[ao]([{C}]*)$")

In [367]:
V = "aeiouyåėęěųȯ"
C = "".join(c.strip() for c in "a b c d e f g h i j k l m n o p r s t u v y z å ć č ď ė ę ě ľ ń ŕ ś š ť ų ź ž ȯ ʒ đ" if c not in V)

C

'bcdfghjklmnprstvzćčďľńŕśšťźžʒđ'

In [429]:
def get_vowels(word):
    return "".join(c for c in word if c in V + "ᵃ")

def get_cons(word):
    return "".join(c for c in word if c not in V + "ᵃ")

In [16]:

possible_prefixes = {
    "ne", "bez", "naj",
    "do", "iz", "na", "nad", "ne", "o", "ob", "obez", "od", "po", "pod", 
    "prě", "prěd", "pri", "pro", 
    "raz", "råz",
    "de", 
    "s", "sȯ",
    "so", "su", "vo", "voz", 
    "sų", "u", "v", 
    "vȯz", "vȯ",
    "vy", "za"
}
possible_prefixes = sorted(list(possible_prefixes), key=len, reverse=True)



def bite_all_prefixes_off(word, verb_nest):

    can_continue = True
    prefixes = []

    while can_continue:
        can_continue = False
        for pref in possible_prefixes:
            if word.startswith(pref):
                # print(pref, word, word[len(pref):])
                new_word = word[len(pref):]
                if new_word.endswith(verb_nest):
                    word = new_word
                    prefixes.append(pref)
                    can_continue = True
                    break
    return word, "’".join(prefixes) + "’"


In [643]:

possible_suffixes = ["iti", "ěti", 'nųti', 'ov', 'ěv', 'yvati', 'ati', 'j', YI, 'ivati', 'jstv']

possible_suffixes = sorted(list(possible_suffixes), key=len, reverse=True)

possible_suffixes = ['eva', 'va', 'vy', 'nų', 've', 'va', "av", 'ov', 'ěv', 'jn', 'ě', 'a', 'j', YI, NASAL, 'i']


def bite_all_suffixes_off(word, verb_nest, suff_overwrite=None):

    if " " in word:
        word = word.split(" ")[0]
        
    if word.endswith("ti"):
        word = word[:-2]
        suffixes = ['ti']
    else:
        return word, ""

    can_continue = True

    if suff_overwrite:
        cur_possible_suffixes = suff_overwrite
    else:
        cur_possible_suffixes = possible_suffixes
    while can_continue:
        can_continue = False
        for pref in cur_possible_suffixes:
            if word.endswith(pref):
                new_word = word[:-len(pref)]
                if new_word.startswith(verb_nest):
                    word = new_word
                    suffixes.append(pref)
                    can_continue = True
                    break
    return word, "+".join(reversed(suffixes)) + "+"



In [ ]:
def insert_YI(word):
    
    base, end = word[:-5], word[-5:]
    end2 = (end
           .replace("žđati", "h" + YI + "ati")
           .replace("šćati", "sk" + YI + "ati")
    )
    if end2 != end:
        return base + end2
    base, end = word[:-4], word[-4:]

    end = (end
           .replace("šati", "s" + YI + "ati")
           .replace("šiti", "h" + YI + "iti")
           # .replace("žati", "g" + YI + "ati")
           .replace("žati", "z" + YI + "ati")
           .replace("čati", "k" + YI + "ati")
           # .replace("čati", "c" + YI + "ati")
           .replace("đati", "d" + YI + "ati")
           .replace("ćati", "t" + YI + "ati")
    )

    return base + end


In [ ]:
izvěstiti {'t'} {'t', 'ć'} {'ě'}
('izvěst', 'izvěšt')


In [410]:
verb_prefixes.query("base_verb == 'izvěstiti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4341,3688,izvěstiti,NaN,v.tr. pf.,"inform, report",NaN,verb,?,?,izvěstiti,izvěstiti,izvěst,?ěstiti?,|izvěsťje|izvěstnik,,,,,0
4348,3692,izvěšćati,NaN,v.tr. ipf.,"inform, report",NaN,verb,?,?,izvěstiti,izvěšćati,izvěšć,?ěšćati?,,,,,,0


In [831]:
def insert_YI(word, signature):

    if " " in word:
        word = word.split(" ")[0]

    if word[-3:] == "ęti":
        word = word[:-3] + "in" + NASAL + "ti"
    if "trim" not in word:
        word = word.replace("idti", "jdti").replace("imati", "jmati")
    if signature == {'s', 't'}:
        if word[-3:] == "sti":
            # rasti -> rastti
            word = word[:-3] + "stti"
        
    base, end = word[:-5], word[-5:]
    
    # jehati
    if signature == {'đ', 'h'}:
        end = end.replace("žđati", "h" + YI + "ati")
    
    if signature == {'ć', 's'}:
        end = end.replace("šćati", "st" + YI + "ati")
    if signature == {'ć', 't'}:
        end = end.replace("šćati", "st" + YI + "ati")
        end = end.replace("ćati", "t" + YI + "ati")
    if signature == {'ć', 'č'}:
        end = end.replace("čiti", "ćiti")
        end = end.replace("čati", "ćati")

    if signature == {'đ', 'd'}:
        end = end.replace("žđati", "zd" + YI + "ati")
        end = end.replace("đati", "d" + YI + "ati")

    # puskati?
    if signature == {'ć', 'k'}:
        end = end.replace("šćati", "sk" + YI + "ati")
    if signature == {'k', 'č'}:
        base, end = word[:-6], word[-6:]
        end = end.replace("čiti", "k" + YI + "iti").replace("čivati", "k" + YI + "ivati")
    

    # pisati
    if signature == {'š', 's'}:  
        end = end.replace("šati", "s" + YI + "ati")
        end = end.replace("šiti", "s" + YI + "iti")
        
    if signature == {'š', 'h'}:  
        end = end.replace("šiti", "h" + YI + "iti")
    if signature == {'š'}:
        base, end = word[:-6], word[-6:]
        end = end.replace("šiti", "h" + YI + "iti")
        end = end.replace("šati", "h" + YI + "ati")
        end = end.replace("šivati", "h" + YI + "ivati")
        

    if signature == {'č', 'k'}:
        end = end.replace("čati", "k" + YI + "ati")
    if signature == {'ž', 'z'}:
        end = end.replace("žati", "z" + YI + "ati")
    if signature == {'ž', 'g'}:
        base, end = word[:-6], word[-6:]

        end = end.replace("žati", "g" + YI + "ati")
        end = end.replace("žiti", "g" + YI + "iti")
        end = end.replace("živati", "g" + YI + "ivati")

    return base + end

In [362]:
# old_stem = verb_prefixes["_stem"]

In [592]:
old_stem2 = verb_prefixes["_stem"]

In [593]:
verb_prefixes['_prefix'] = ''
verb_prefixes['_stem'] = ''
verb_prefixes['_suffix'] = ''
verb_prefixes['_is_irregular'] = 0


In [594]:
verb_prefixes.loc[
    verb_prefixes.query("base_verb.str.contains('byti') and not isv.str.contains('byti')").index,
    "_is_irregular"
] = 1
verb_prefixes.loc[
    verb_prefixes.query("base_verb == 'dějati' and isv.str.len() > 10").index,
    "_is_irregular"
] = 1

verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('čuvstvo')").index,
    "_is_irregular"
] = 1

verb_prefixes.loc[
    verb_prefixes.query("(en.str.contains('sleep') or en.str.contains('dream')) and isv.str.contains('sn')").index,
    "_is_irregular"
] = 1
verb_prefixes.loc[
    verb_prefixes.query("(en.str.contains('sleep') or en.str.contains('dream')) and isv.str.contains('sn')").index,
    "base_verb"
] = 'sȯniti'
verb_prefixes.loc[
    verb_prefixes.query("isv == 'velěti'").index,
    "_is_irregular"
] = 1

verb_prefixes.loc[
    verb_prefixes.query("isv == 'vměsati'").index,
    "_is_irregular"
] = 1


 


In [637]:
verb_prefixes.loc[
    verb_prefixes.query("isv == 'velěti'").index,
    "_is_irregular"
] = 1

In [595]:
from collections import Counter
signatures_cnt = Counter()

In [619]:
g = verb_prefixes.query('_stem == "" and genesis != genesis and base_verb == "staviti"')

In [621]:
k = 'staviti'
if k == "" or len(g.isv.unique()) == 1:
    print(1)
if all(k in member for member in g.isv.unique()):
    print(2)


2


In [881]:
verb_prefixes.to_pickle("attempt4.pkl")

In [826]:
verb_prefixes = pd.read_pickle("attempt3.pkl")

In [767]:
len(verb_prefixes), len(verb_prefixes.query('_stem != ""'))

(4985, 4882)

In [723]:
N = 0

for k, g in verb_prefixes.query("_stem == '' and _is_irregular == 0").groupby("base_noun"):
    if k == "":
        continue
    
    LSC, RSC = zip(
        *g.isv.apply(lambda x:
            bite_all_suffixes_off(x.replace("?", ""), "")
       )
    )
    
    last_cons = {get_cons(l)[-1:] for l in list(LSC) + [k]}
    if len(last_cons) == 2 or any(l[-1:] == 'ę' for l in LSC) or k in ['idti', 'imati', 'råsti']:
        LSC, RSC = zip(
            *g.isv.apply(lambda x:
                bite_all_suffixes_off(insert_YI(x.replace("?", ""), last_cons), "")
           )
        )
        
    for ((i, raw_isv), lsc, rsc) in zip(
                g.isv.apply(lambda x: insert_YI(x, last_cons)).items(),
                LSC,
                RSC
        ):
            local_stem, local_pref = bite_all_prefixes_off(lsc, "")
            local_stem = k.replace("†", "")
            if "|" in local_stem:
                local_stem = local_stem.split("|")[-1]
            if local_stem[-1] in {'a', 'y', 'j', 'o'}:
                local_stem = local_stem[:-1]
            if k == "dopis":
                local_stem = "pis"
            if k == "prězir":
                local_stem = "zir"
            if k == "slovjanin":
                local_stem = "slovjan"
            if k == "smråd":
                local_stem = "smŕd"
            if k.startswith("råz"):
                local_stem = local_stem[3:]
                
            if raw_isv == "vojevati":
                local_stem = "voj"
            if raw_isv == "valjcevati":
                local_stem = "valjc"
                
            if local_stem not in raw_isv:
                local_stem = local_stem.replace("ŕ", "r").replace("ť", "t")

            # print(raw_isv, f"({lsc}, {rsc})", "=", raw_isv.partition(local_stem))
            tmp_part = raw_isv.partition(local_stem)
            if tmp_part[1]:
                print(raw_isv, "=", tmp_part, f"({k})")
                verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix']] = tmp_part
            else:
                print(raw_isv, "!=", tmp_part, f"({k}, {local_stem})")


len(verb_prefixes), len(verb_prefixes.query('_stem != ""'))

sebesmŕtiti = ('sebe', 'smŕt', 'iti') (smŕt)
valjcevati = ('', 'valjc', 'evati') (val)
valiti = ('', 'val', 'iti') (val|vala)
valjati = ('', 'val', 'jati') (val|vala)
valjati = ('', 'val', 'jati') (val|vala)
vlastniti = ('', 'vlast', 'niti') (vlasť)
vojevati = ('', 'voj', 'evati') (vojna)
učęstvovati = ('u', 'čęst', 'vovati') (čęsť)
obagriti = ('o', 'bagr', 'iti') (†bagry)


(4985, 4864)

In [718]:
tmp_df = verb_prefixes.query("isv.str.contains('(valjcevati)|(sebesmŕtiti)|(vlastniti)|(učęstvovati)')")

for i, row in tmp_df.iterrows():
    verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["", "", "", 0]


C:\Users\79165\AppData\Local\Temp\ipykernel_9748\4139924804.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = verb_prefixes.query("isv.str.contains('(valjcevati)|(sebesmŕtiti)|(vlastniti)|(učęstvovati)')")


In [830]:
verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('slu[šh]')").index,
    "base_verb"
] = 'slušati'


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4265,36533.0,izslušati,NaN,v.tr. pf.,"hear out, listen to",NaN,verb,?,?,izslušati,izslušati,izsluš,slušati,,,,izsluš,ati,0
4266,36532.0,izslušivati,NaN,v.tr. ipf.,"hear out, listen to",NaN,verb,?,?,izslušati,izslušivati,izsluš,?lušivati?,,,,izsluš,ivati,0
7465,28200.0,naslušati,NaN,v.refl. pf.,"hear a lot of, hear enough of",NaN,verb,?,?,slušati,naslušati sę,nasluš,slušati,,,na,sluš,ati,0
7615,21368.0,ne poslušati,NaN,v.tr. pf.,disobey,NaN,verb,?,?,slušati,ne poslušati,ne posluš,? poslušati?,,,,,,0
7617,21367.0,ne slušati,NaN,v.tr. ipf.,disobey,NaN,verb,?,?,slušati,ne slušati,ne sluš,? slušati?,,,,,,0
10463,10769.0,podslušati,NaN,v.tr. pf.,"eavesdrop, wiretap",NaN,verb,?,?,podslušati,podslušati,podsluš,slušati,,,,podsluš,ati,0
10466,22778.0,podslušivati,NaN,v.tr. ipf.,"eavesdrop, wiretap",NaN,verb,?,?,podslušati,podslušivati,podsluš,?lušivati?,,,,podsluš,ivati,0
10894,71.0,poslušati,NaN,v.tr. pf.,"listen, obey",NaN,verb,?,?,slušati,poslušati,posluš,slušati,,,po,sluš,ati,0
11529,5991.0,prěslušati,NaN,v.tr. pf.,interrogate,NaN,verb,?,?,prěslušati,prěslušati,prěsluš,slušati,,,,prěsluš,ati,0
11531,5990.0,prěslušivati,NaN,v.tr. ipf.,interrogate,NaN,verb,?,?,prěslušati,prěslušivati,prěsluš,?lušivati?,,,,prěsluš,ivati,0


In [ ]:
verb_prefixes = pd.read_pickle("attempt3.pkl")

In [891]:
N = 0

for k, g in verb_prefixes.query("_stem == '' and _is_irregular == 0").groupby("base_verb"):
    if k == "":
        continue
    k = k.replace("ne ", "")
    g = verb_prefixes.query("base_verb == @k").copy()
    g.isv = g.isv.str.replace("ne ", "")
    
    # print()
    risky_try = False
    #RSC = [rsc.replace("?", "") for rsc in g.right_stem_cand.values.tolist()]
    #LSC = g.left_stem_cand.values.tolist()

    LSC, RSC = zip(
        *g.isv.apply(lambda x:
            bite_all_suffixes_off(x.replace("?", ""), "")
       )
    )
    
    last_cons = {get_cons(l)[-1:] for l in LSC}
    if len(last_cons) == 2 or any(l[-1:] == 'ę' for l in LSC) or k in ['idti', 'imati', 'råsti', 'slušati']:
        LSC, RSC = zip(
            *g.isv.apply(lambda x:
                bite_all_suffixes_off(insert_YI(x.replace("?", ""), last_cons), "")
           )
        )
    last_vowels = {get_vowels(l)[-1:] for l in LSC}
    last_cons2 = {get_cons(l)[-1:] for l in LSC}
    if len(last_cons) > 1:
        signatures_cnt["_".join(last_cons)] += 1
        if False:
            print()
            print(k, LSC)
            print(RSC)
            print(last_cons)
            print(last_cons2)
            print(common_suffix(LSC))
            print(last_vowels)
        de_prefixed_stem = Counter([bite_all_prefixes_off(l, "")[0] for l in LSC])
        stem_cons = {get_cons(l) for l in de_prefixed_stem}
        print(de_prefixed_stem, stem_cons)



    if last_vowels == {'a', 'o'}:
        LSC = [AO_RE.sub(f"{AO}\\1", l) for l in LSC]
    if (k == 'råsti' or k == 'glåsiti') and last_vowels == {'å', 'a'}:
        print(last_vowels)
        LSC = [AO_RE.sub(f"å\\1", l) for l in LSC]


    #     if "†" in k:
    #         LSC += [
    #             bite_all_suffixes_off(
    #                 k.replace("†", ""), 
    #                 "")[0]
    #         ]

    # true_stem = list({s for s in LSC if all(rsc.startswith(s) for rsc in RSC)})
    true_stem = common_suffix(LSC)
    
    if k == 'biti':
        true_stem = "bi"
    if k == 'dati':
        true_stem = "da"
    if k == 'děti':
        true_stem = "dě"
    if k == 'vyti':
        true_stem = "vy"

    if k == 'kovati':
        true_stem = "kov"
    if k == 'bajati':
        true_stem = "baj"
    if k == 'vajati':
        true_stem = "vaj"
    if k == 'liti':
        true_stem = "li"
    if k == 'gnųti':
        true_stem = "gn"
    if k == '†gynųti':
        true_stem = "gyn"
    if k == 'tajiti':
        true_stem = "taj"
    if k == 'sȯsati':
        true_stem = "sȯs"
    if k == 'loviti':
        true_stem = "lov"
    if k == 'pojiti':
        true_stem = "pꜵj"
    if k == 'sějati':
        true_stem = "sěj"
    if k == 'dějati':
        true_stem = "děj"
    if k == 'dějati':
        true_stem = "děj"
    if k == 'zějati':
        true_stem = "zěj"
    if k == 'daviti' or k == 'davati':
        true_stem = "dav"
        
    
    de_prefixed_stem = Counter([bite_all_prefixes_off(l, "")[0] for l in LSC])
    stem_cons = {get_cons(l) for l in de_prefixed_stem}
    if len(true_stem) <= 1:
        #print()
        if False:
            true_stem = k[:-3]
            risky_try = True
            #print("TRY: ", true_stem, k, k[:-3])
            #print(de_prefixed_stem, stem_cons)
            #print(last_cons2, last_cons)
            #print(LSC)


            # continue
            pass

    if (len(true_stem) <= 1 and len(de_prefixed_stem) > 1 and len(stem_cons) == 1) or risky_try:
        if get_cons(k) in stem_cons:
            print("IN")
        #print(de_prefixed_stem)
        # print(stem_cons)
        # print(k, LSC)
        for ((i, raw_isv), lsc, rsc) in zip(
                g.isv.apply(lambda x: insert_YI(x, last_cons)).items(),
                LSC,
                RSC
        ):
            local_stem, local_pref = bite_all_prefixes_off(lsc, "")
            if risky_try:
                local_stem = true_stem
                if true_stem == "brån" and true_stem not in lsc:
                    local_stem = "bran"
            if k == 'slati' or k == '†strěti':
                local_stem = "s" + local_stem
            if k == "klåti" and raw_isv == "råzkalati":
                local_stem = "kala"
            if k == "krojiti":
                local_stem = raw_isv[-7:-3]                
            if k == 'zvati' or k == "rvati" or k == "kovati":
                local_stem = local_stem + "v"
            if k == "sȯhnųti":
                local_stem = "sȯh" if "sȯh" in raw_isv else "syh"
            if local_stem[0] == 'ȯ':
                local_stem = local_stem[1:]
                

            # print(raw_isv, f"({lsc}, {rsc})", "=", raw_isv.partition(local_stem))
            print(raw_isv, "=", raw_isv.partition(local_stem))
            verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix']] = raw_isv.partition(local_stem)
            if risky_try:
                verb_prefixes.loc[i, ['_is_irregular']] = -1

        continue

    if len(true_stem) <= 1:
        print()
        
        print(true_stem)
        print(k, LSC)
        print(k, last_cons2, last_cons, last_vowels)
        #print(LSC)
        #print(de_prefixed_stem)
        #print({get_cons(l) for l in de_prefixed_stem})
        
        N += 1
        #if N == 5:
        #    break
        continue
    

    for i, row in g.isv.apply(lambda x: split_carefully(insert_YI(x, last_cons), true_stem)).items():
        verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix']] = row[0], true_stem, row[2]
        raw_isv = g.loc[i, 'isv']
        print(raw_isv, "=", row[0], true_stem, row[2])


len(verb_prefixes), len(verb_prefixes.query('_stem != ""'))

(4985, 4959)

In [893]:
verb_prefixes.query('_stem == ""').shape

# verb_prefixes.query('base_verb == "vlěkti"')


(26, 19)

In [894]:
verb_prefixes.to_pickle("verb_prefixes_last.pkl")

In [914]:
verb_prefixes._suffix = verb_prefixes._suffix.str.replace(YI, "⒥").str.replace(NASAL, "⒩")

verb_prefixes[['id', 'isv_orig', 'addition', 'partOfSpeech', 'en', 'genesis',
       'base_verb', 'derived_nouns', 'base_noun', '_prefix', '_stem',
       '_suffix', '_is_irregular']].to_csv('verb_morphemes.csv')

In [932]:
verb_prefixes._suffix.value_counts()

iti          1312
ati          1213
vati          626
ti            401
jati          313
nųti          218
ěti           206
⒥ati          172
avjati         60
aviti          55
ivati          51
⒩ti            42
jiti           34
avati          30
               28
ěvati          26
ovati          25
vyvati         16
yvati          14
ųti            13
oviti          12
⒥iti           11
evati          10
ějati           9
vti             9
ajati           8
izovati         7
jnjati          5
jniti           5
stti            5
⒥ivati          4
tati            4
in⒩ti           3
niti            3
ějivati         3
ijati           2
njati           2
inųti           2
jevati          2
amjati          2
omiti           2
ajivati         2
ipravjati       2
ipraviti        2
jivati          1
vovati          1
čiti            1
ropivati        1
ropiti          1
opivati         1
ěpravjati       1
ěpraviti        1
kovati          1
opiti           1
ęti             1
iveti     

In [939]:

verb_prefixes.query("isv.str.contains('vědom')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
9700,23887.0,osvědomiti,NaN,v.refl. pf.,inquire,NaN,verb,?,?,věděti,osvědomiti sę,osvědom,?ědomiti?,,,os,věd,omiti,-1
15985,1619.0,uvědomiti,NaN,v.tr. pf.,"inform, notify",NaN,verb,?,?,věděti,uvědomiti,uvědom,?ědomiti?,,,u,věd,omiti,-1


In [940]:
verb_prefixes.query("_suffix.str.contains('prav') or _suffix.str.contains('rop') or _suffix.str.contains('amjati')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
9699,23888.0,osvědamjati,NaN,v.refl. ipf.,inquire,NaN,verb,?,?,věděti,osvědamjati sę,osvědam,?ědamjati?,,,os,věd,amjati,-1
11489,3539.0,prěpraviti,NaN,v.tr. pf.,cross,NaN,verb,?,?,praviti,prěpraviti,prěprav,praviti,,,,pr,ěpraviti,0
11490,3528.0,prěpravjati,NaN,v.tr. ipf.,cross,NaN,verb,?,?,praviti,prěpravjati,prěprav,?pravjati?,,,,pr,ěpravjati,0
11795,21073.0,pripraviti,NaN,v.tr. pf.,"season, flavour",NaN,verb,?,?,praviti,pripraviti,priprav,praviti,,,,pr,ipraviti,0
11796,21075.0,pripraviti,NaN,v.tr. pf.,prepare,NaN,verb,?,?,praviti,pripraviti,priprav,praviti,,,,pr,ipraviti,0
11797,21076.0,pripravjati,NaN,v.tr. ipf.,season,NaN,verb,?,?,praviti,pripravjati,priprav,?pravjati?,,,,pr,ipravjati,0
11798,21078.0,pripravjati,NaN,v.tr. ipf.,prepare,NaN,verb,?,?,praviti,pripravjati,priprav,?pravjati?,,,,pr,ipravjati,0
12100,35765.0,propiti,NaN,v.tr. pf.,spend on alcohol,NaN,verb,?,?,piti,propiti,prop,piti,,,,p,ropiti,-1
12101,35764.0,propivati,NaN,v.tr. ipf.,spend on alcohol,NaN,verb,?,?,piti,propivati,prop,opivati,,,,p,ropivati,-1
15982,1620.0,uvědamjati,NaN,v.tr. ipf.,"inform, notify",NaN,verb,?,?,věděti,uvědamjati,uvědam,?ědamjati?,,,u,věd,amjati,-1


In [936]:
verb_prefixes.query("_suffix.str.contains('⒩') or base_verb.str.contains('ęti')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4025,5514.0,izjęti,(izȯjme),v.tr. pf.,take out,NaN,verb,?,?,jęti,izjęti,izjęti,jęti,,,iz,jin,⒩ti,0
4125,23316.0,iznajęti,(iznajme),v.tr. pf.,"rent out, let (to sbd.)",NaN,verb,?,?,jęti,iznajęti,iznajęti,jęti,,,izna,jin,⒩ti,0
4395,2932.0,izžęti,(izžne),v.tr. pf.,reap,NaN,verb,?,?,žęti,izžęti,izžęti,žęti,,,iz,žin,⒩ti,0
4595,16800.0,jęti,(jme),v.tr. ipf.,take,NaN,verb,?,?,jęti,jęti,jęti,jęti,,,,jin,⒩ti,0
4948,17655.0,klęti,(klne),v.intr. ipf.,"curse, swear",NaN,verb,?,?,klęti,klęti,klęti,klęti,,,,klin,⒩ti,0
6570,27576.0,męti,(mne),v.tr. ipf.,crumple,NaN,verb,?,?,męti,męti,męti,męti,,,,min,⒩ti,0
7080,1502.0,načęti,(načne),v.refl. pf.,"begin (intr.), start (intr.)",NaN,verb,?,?,načinati,načęti sę,načęti,?čęti?,,,,način,⒩ti,0
7081,22780.0,načęti,(načne),v.tr. pf.,"begin (tr.), start (tr.)",NaN,verb,?,?,načinati,načęti,načęti,?čęti?,,,,način,⒩ti,0
7196,9410.0,najęti,(najme),v.tr. pf.,"hire, rent (from sbd.), lease",NaN,verb,?,?,jęti,najęti,najęti,jęti,,,na,jin,⒩ti,0
7197,17861.0,najęti,(najme),v.tr. pf.,employ,NaN,verb,?,?,jęti,najęti,najęti,jęti,,,na,jin,⒩ti,0


In [934]:
verb_prefixes.query("_suffix == 'in⒩ti' or base_verb == 'počęti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8939,5605.0,odpočęti,(odpočne),v.intr. pf.,"rest, relax",NaN,verb,?,?,počęti,odpočęti,odpočęti,počęti,,,od,poč,in⒩ti,-1
8941,3295.0,odpočivati,NaN,v.intr. ipf.,"rest, relax",NaN,verb,?,?,počęti,odpočivati,odpoč,?čivati?,,,od,poč,ivati,-1
10306,22788.0,počęti sę,(počne),v.tr. pf.,"begin (intr.), start (intr.)",NaN,verb,?,?,počęti,počęti sę,počęti,?čęti sę?,,,,poč,ęti,-1
10307,1078.0,počęti,(počne),v.tr. pf.,"begin (tr.), start (tr.), commence",NaN,verb,?,?,počęti,počęti,počęti,?čęti?,,,,poč,in⒩ti,-1
17386,32371.0,započęti,(započne),v.tr. pf.,"initiate, commence, kick off, originate, start up",NaN,verb,?,?,počęti,započęti,započęti,počęti,,,za,poč,in⒩ti,-1


In [913]:
vc = verb_prefixes._prefix.value_counts()
vc[vc == 3]

prěo      3
vȯzs      3
sra       3
ne        3
ra        3
obez      3
prěs      3
råzpro    3
pora      3
prědpo    3
Name: _prefix, dtype: int64

In [917]:
verb_prefixes.query("_suffix.str.contains('tt')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8933,29148.0,odplesti,(odplete),v.tr. pf.,"unweave, unplait, unbraid, untwine",NaN,verb,?,?,plesti,odplesti,odplesti,plesti,,,od,ple,stti,-1
10237,5019.0,plesti,(plete),v.tr. ipf.,"braid, knit, plait, weave",NaN,verb,?,?,plesti,plesti,plesti,plesti,,,,ple,stti,-1
12772,34338.0,råzplesti,(råzplete),v.tr. pf.,"untwine, unweave, unbraid, unplait",NaN,verb,råz’,plesti,plesti,råzplesti,råzplesti,plesti,,,råz,ple,stti,-1
14207,34330.0,splesti,(splete),v.tr. pf.,"braid, knit, plait, weave",NaN,verb,?,?,plesti,splesti,splesti,plesti,,,s,ple,stti,-1
16806,20272.0,vplesti,(vplete),v.tr. pf.,weave into,NaN,verb,?,?,plesti,vplesti,vplesti,plesti,,,v,ple,stti,-1


In [924]:
verb_prefixes.query("_stem.str.contains(@AO)")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
2041,25285.0,dogarjati,NaN,v.tr. ipf.,burn down,NaN,verb,?,?,gorěti,dogarjati,dogar,?garjati?,,,do,gꜵr,jati,0
2045,14534.0,dogoditi,NaN,v.refl. pf.,happen,NaN,verb,?,?,goditi,dogoditi sę,dogod,goditi,,,do,gꜵd,iti,0
2046,31826.0,dogorěti,NaN,v.tr. pf.,burn down,NaN,verb,?,?,gorěti,dogorěti,dogor,gorěti,,,do,gꜵr,ěti,0
2047,5534.0,dogovarjati,NaN,v.refl. ipf.,agree,NaN,verb,?,?,govoriti,dogovarjati sę,dogovar,?govarjati?,,,do,govꜵr,jati,0
2048,2899.0,dogovarjati,NaN,v.tr. ipf.,"negotiate, contract, settle, agree upon",NaN,verb,?,?,govoriti,dogovarjati,dogovar,?govarjati?,,,do,govꜵr,jati,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17506,31567.0,zatknųti,NaN,v.tr. pf.,"plug, stop up",NaN,verb,?,?,tknųti,zatknųti,zatk,tknųti,,,z,ꜵtk,nųti,0
17513,14183.0,zatopiti,NaN,v.tr. pf.,"flood, inundate",NaN,verb,?,?,topiti,zatopiti,zatop,topiti,,,za,tꜵp,iti,0
17521,5204.0,zatvarjati,NaN,v.tr. ipf.,"close, shut",NaN,verb,?,?,tvoriti,zatvarjati,zatvar,?tvarjati?,,,za,tvꜵr,jati,0
17524,5205.0,zatvoriti,NaN,v.tr. pf.,"close, shut",NaN,verb,?,?,tvoriti,zatvoriti,zatvor,tvoriti,|zatvorka,,za,tvꜵr,iti,0


In [926]:
verb_prefixes.query("_suffix == ''")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
130,24156.0,aklimatizovati,NaN,v.intr. ipf.,acclimatize,I,verb,?,?,aklimatizovati,aklimatizovati,aklimatiz,aklimatizovati,|aklimatizacija,,akljmatizovati,aklimatizo,,0
1185,22979.0,bųde/bųdųt,NaN,v.aux. ipf.,there will be,NaN,verb,?,?,,bųde/bųdųt,bųde/bųdųt,?bųde/bųdųt?,,,,,,0
4297,5567.0,izsysati,NaN,#v.tr. ipf.,suck out,NaN,None,?,?,sȯsati,izsysati,izsys,?ysati?,,,izsysati,sȯs,,0
4473,8245.0,"je, jest",NaN,v.ipf.,there is,NaN,verb,?,?,,"je, jest","je, jest","?je, jest?",,,,,,0
4592,14605.0,jestvovati,NaN,v.intr. ipf.,exist,NaN,verb,?,?,byti,jestvovati,jestv,jestvovati,,,,,,1
5287,3519.0,"kopijovati, kopirovati",NaN,v.tr. ipf.,copy,I,verb,?,?,,"kopijovati, kopirovati","kopijovati, kopir","?kopijovati, kopirovati?",,,,,,0
9061,36666.0,odsųtstvovati,NaN,v.intr. ipf.,"be absent, be away",NaN,verb,?,?,byti,odsųtstvovati,odsųtstv,sųtstvovati,,,,,,1
9155,29304.0,"ogluhnųti, oglušeti",NaN,v.intr. pf.,become deaf,NaN,verb,?,?,,"ogluhnųti, oglušeti","ogluhnųti, oglušeti","?gluhnųti, oglušeti?",,,,,,0
11859,23860.0,prisųtstvovati pri,(+6),v.intr. ipf.,"attend, be present at",NaN,verb,?,?,byti,prisųtstvovati pri,prisųtstvovati pri,?tstvovati pri?,,,,,,1
12189,5479.0,protivdějati,NaN,v.intr. ipf.,counteract,NaN,verb,?,?,dějati,protivdějati,protivdě,?tivdějati?,,,,,,1


In [930]:
WTF = vc[vc < 3].index

verb_prefixes.query("_prefix in @WTF")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
130,24156.0,aklimatizovati,NaN,v.intr. ipf.,acclimatize,I,verb,?,?,aklimatizovati,aklimatizovati,aklimatiz,aklimatizovati,|aklimatizacija,,akljmatizovati,aklimatizo,,0
784,36698.0,bezpokojiti,NaN,v.refl. ipf.,"worry, be anxious, feel uneasy",NaN,verb,?,?,†pokojiti,bezpokojiti sę,bezpoko,?kojiti?,,,bez,poko,jiti,0
785,19654.0,bezpokojiti,NaN,v.tr. ipf.,"disturb, trouble, upset, bother",NaN,verb,?,?,†pokojiti,bezpokojiti,bezpoko,?kojiti?,,,bez,poko,jiti,0
2185,7213.0,doråzuměti,NaN,v.refl. pf.,communicate,NaN,verb,?,?,uměti,doråzuměti sę,doråzum,uměti,,,dora,zum,ěti,0
2253,22119.0,dotknųti,NaN,v.refl. pf.,touch,NaN,verb,?,?,tknųti,dotknųti sę,dotk,tknųti,,,d,ꜵtk,nųti,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17397,21659.0,zapoznati,NaN,v.tr. pf.,acquaint,NaN,verb,?,?,znati,zapoznati,zapozn,znati,,,zapo,zn,ati,0
17463,23843.0,zasråmiti,NaN,v.tr. pf.,embarrass,NaN,verb,?,?,sråmiti,zasråmiti,zasråm,sråmiti,,,zas,råm,iti,0
17464,23842.0,zasramjati,NaN,v.tr. ipf.,embarrass,NaN,verb,?,?,sråmiti,zasramjati,zasram,?ramjati?,,,zas,ram,jati,0
17505,33093.0,zatknųti,NaN,v.refl. pf.,be stuck,NaN,verb,?,?,tknųti,zatknųti sę,zatk,tknųti,,,z,ꜵtk,nųti,0


In [931]:
verb_prefixes.query("base_verb == 'tknųti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
2253,22119.0,dotknųti,NaN,v.refl. pf.,touch,NaN,verb,?,?,tknųti,dotknųti sę,dotk,tknųti,,,d,ꜵtk,nųti,0
2254,481.0,dotknųti,NaN,v.tr. pf.,touch,NaN,verb,?,?,tknųti,dotknųti,dotk,tknųti,,,d,ꜵtk,nųti,0
14248,22123.0,spotknųti,NaN,v.refl. pf.,stumble,NaN,verb,?,?,tknųti,spotknųti sę,spotk,tknųti,,,sp,ꜵtk,nųti,0
17505,33093.0,zatknųti,NaN,v.refl. pf.,be stuck,NaN,verb,?,?,tknųti,zatknųti sę,zatk,tknųti,,,z,ꜵtk,nųti,0
17506,31567.0,zatknųti,NaN,v.tr. pf.,"plug, stop up",NaN,verb,?,?,tknųti,zatknųti,zatk,tknųti,,,z,ꜵtk,nųti,0


In [918]:
AO

'ꜵ'

In [916]:
vc.to_csv("observed_verb_prefixes.csv")

In [911]:
verb_prefixes.query("isv.str.contains('aklimatizovati')")
verb_prefixes.loc[130, ]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
130,24156.0,aklimatizovati,NaN,v.intr. ipf.,acclimatize,I,verb,?,?,aklimatizovati,aklimatizovati,aklimatiz,aklimatizovati,|aklimatizacija,,akljmatizovati,aklimatizo,,0


In [906]:
verb_prefixes._suffix.value_counts()

iti          1312
ati          1213
vati          626
ti            401
jati          313
nųti          218
ěti           206
⒥ati          172
avjati         60
aviti          55
ivati          51
⒩ti            42
jiti           34
avati          30
               28
ěvati          26
ovati          25
vyvati         16
yvati          14
ųti            13
oviti          12
⒥iti           11
evati          10
ějati           9
vti             9
ajati           8
izovati         7
jnjati          5
jniti           5
stti            5
⒥ivati          4
tati            4
in⒩ti           3
niti            3
ějivati         3
ijati           2
njati           2
inųti           2
jevati          2
amjati          2
omiti           2
ajivati         2
ipravjati       2
ipraviti        2
jivati          1
vovati          1
čiti            1
ropivati        1
ropiti          1
opivati         1
ěpravjati       1
ěpraviti        1
kovati          1
opiti           1
ęti             1
iveti     

In [ ]:
все глаголы изкльучајучи 26 много нерегуларных имајут оззначенје префиксов, суфиксов и стема-основы

In [740]:
def manual_insert(base_verb, true_stem, last_cons=set(), dry_run=True):
    g = verb_prefixes.query("base_verb == @base_verb")
    for i, row in g.isv.apply(lambda x: split_carefully(insert_YI(x, last_cons), true_stem)).items():
        if not dry_run:
            verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix']] = row[0], true_stem, row[2]
        raw_isv = g.loc[i, 'isv']
        print(raw_isv, "=", row[0], true_stem, row[2])

manual_insert("uzdråviti", "zdråv", dry_run=False)

uzdråviti = u zdråv iti
uzdravjati = u zdråv jati


In [745]:
manual_insert("občiti", "obć", {'ć', 'č'}, dry_run=False)


občiti s =  obć iti
občiti =  obć iti
sȯobćati = sȯ obć ati
sȯobćati = sȯ obć ati
sȯobčiti = sȯ obć iti


In [756]:
manual_insert("kajati", "kaj", {}, dry_run=False)


kajati =  kaj ati
pokajati = po kaj ati
råzkajati = raz kaj ati


In [774]:
verb_prefixes.query("base_verb == '†spěti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
11830,21476.0,prispěti,NaN,v.intr. pf.,arrive on time,NaN,verb,?,?,†spěti,prispěti,prisp,?pěti?,,,pri,sp,ěti,0
15872,4042.0,uspěti,NaN,v.intr. pf.,"succeed, manage (to do sth.), make it, be succ...",NaN,verb,?,?,†spěti,uspěti,usp,?pěti?,,,u,sp,ěti,0


In [778]:
verb_prefixes.loc[[11831, 15873], "base_verb"] = "†spěti"
manual_insert("†spěti", "spě", {}, dry_run=False)


prispěti = pri spě ti
prispěvati = pri spě vati
uspěti = u spě ti
uspěvati = u spě vati


In [780]:
verb_prefixes.query("_stem == '' and base_verb == 'pěvati' and _is_irregular == 0")

manual_insert("pěvati", "pě", {}, dry_run=False)


pěvati =  pě vati
spěvati = s pě vati


In [782]:
manual_insert("tajati", "taj", {}, dry_run=False)


odtajati = od taj ati
råztajati = raz taj ati
tajati =  taj ati


In [788]:
manual_insert("vŕnųti", "ŕ", {}, dry_run=False)


obŕnųti = ob ŕ nųti
obŕnųti = ob ŕ nųti
obŕnųti = ob ŕ nųti
obŕnųti = ob ŕ nųti
obŕnųti = ob ŕ nųti
povŕnųti = pov ŕ nųti
povŕtati = pov ŕ tati
vŕnųti = v ŕ nųti
vŕnųti = v ŕ nųti


In [790]:
manual_insert("vějati", "věj", {}, dry_run=False)

manual_insert("minųti", "min", {}, dry_run=False)


vějati =  věj ati
minųti =  min ųti
råzminųti = raz min ųti


In [794]:
# verb_prefixes.query("en.str.contains('wind')")
KNOWN_BASES = verb_prefixes.query("base_verb != ''").isv.unique()


In [798]:
verb_prefixes.query("base_verb == 'běgati'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
679,2441.0,běgati,NaN,v.intr. ipf.,run,NaN,verb,?,?,běgati,běgati,běg,běgati,|běgstvo,běg,,běg,ati,0
3907,1647.0,izběgati,NaN,v.intr. ipf.,"escape, flee",NaN,verb,?,?,běgati,izběgati,izběg,běgati,,,iz,běg,ati,0
3908,1953.0,izběgati,(+2),v.intr. ipf.,avoid,NaN,verb,?,?,běgati,izběgati,izběg,běgati,,,iz,běg,ati,0
3909,36474.0,izběgati,NaN,v.intr. ipf.,run out,NaN,verb,?,?,běgati,izběgati,izběg,běgati,,,iz,běg,ati,0
7053,27920.0,naběgati,NaN,v.refl. pf.,run a lot,NaN,verb,?,?,běgati,naběgati sę,naběg,běgati,,,na,běg,ati,0
8292,28731.0,oběgati,NaN,v.tr. pf.,run around,NaN,verb,?,?,běgati,oběgati,oběg,běgati,,,o,běg,ati,0
8708,29040.0,odběgati,NaN,v.intr. ipf.,run off,NaN,verb,?,?,běgati,odběgati,odběg,běgati,,,od,běg,ati,0
12476,30100.0,råzběgati,NaN,v.refl. ipf.,#start running a lot,NaN,verb,råz’,běgati sę,běgati,råzběgati sę,råzběg,běgati,,,raz,běg,ati,0
12477,30101.0,råzběgati,NaN,v.refl. ipf.,"run in all directions, disperse",NaN,verb,råz’,běgati sę,běgati,råzběgati sę,råzběg,běgati,,,raz,běg,ati,0
13410,18677.0,sběgati,NaN,v.refl. ipf.,"coincide, converge",NaN,verb,?,?,běgati,sběgati sę,sběg,běgati,,,s,běg,ati,0


In [872]:
verb_prefixes.query("_stem == '' and _is_irregular == 0 and not isv.str.contains(',') and base_verb == ''")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
1185,22979.0,bųde/bųdųt,NaN,v.aux. ipf.,there will be,NaN,verb,?,?,,bųde/bųdųt,bųde/bųdųt,?bųde/bųdųt?,,,,,,0
11668,36346.0,pričisljati k,(+3),v.tr. ipf.,"number among, rank among",NaN,verb,?,?,,pričisljati k,pričisljati k,?čisljati k?,,,,,,0
16395,2605.0,viniti za,NaN,v.tr. ipf.,"accuse of, blame for",NaN,verb,?,?,,viniti za,viniti za,?initi za?,,,,,,0
16648,23983.0,vȯobražati sobě,NaN,v.tr. ipf.,"imagine, envision",NaN,verb,vȯ’,obražati sobě,,vȯobražati sobě,vȯobražati sobě,?ražati sobě?,,,,,,0


In [878]:
verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('obra[zž]')").index,
    "base_verb"
] = "obraziti"

In [876]:
 
verb_prefixes.loc[
    [11668, 16395],
    "base_verb"
] = ['čisliti', 'viniti']

In [852]:
# uhađati 

verb_prefixes.loc[
    17868,
    ["base_verb", 'derived_nouns']
] = ["žmuriti", "žmurky"]

 
verb_prefixes.loc[
    [15642, 17368, 16148, 17553, 11099],
    "base_verb"
] = ['hoditi', "phati", "děti", 'visěti', 'zrěti']

In [871]:
 
verb_prefixes.loc[
    [2269, 7618, 7619, 8642, 8643, 8706, 8707, 8723, 8724, 12080],
    "base_verb"
] = ['vesti', 'glåsiti', 'glåsiti', 'viniti', 'viniti', 'bočiti', 'bočiti', 'bočiti', 'bočiti', 'myti']

In [848]:
verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('vęz')").index,
    "base_verb"
] = 'vęzati'


In [857]:
verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('voditi') or base_verb == 'rųkovoditi'").index,
    "base_verb"
] = 'voditi'


In [844]:
verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('iměti') and base_verb != 'primětiti'").index,
    "base_verb"
] = "imati"
verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('iměti') and base_verb != 'primětiti'").index,
    "isv"
] = "imati"


In [875]:
verb_prefixes.query("isv.str.contains('primě[ćt]')")
verb_prefixes.query("isv.str.contains('obra[zž]')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4144,771.0,izobraziti,NaN,v.tr. pf.,"depict, portray",NaN,verb,?,?,raziti,izobraziti,izobraz,raziti,,,izob,raz,iti,0
4145,1865.0,izobražati,NaN,v.tr. ipf.,"depict, portray",NaN,verb,?,?,raziti,izobražati,izobraž,?ražati?,,,izob,raz,յati,0
11465,11320.0,prěobraziti,NaN,v.tr. pf.,"morph, transform",NaN,verb,?,?,raziti,prěobraziti,prěobraz,raziti,,,prěob,raz,iti,0
11466,11322.0,prěobražati,NaN,v.tr. ipf.,"morph, transform",NaN,verb,?,?,raziti,prěobražati,prěobraž,?ražati?,,,prěob,raz,յati,0
16648,23983.0,vȯobražati sobě,NaN,v.tr. ipf.,"imagine, envision",NaN,verb,vȯ’,obražati sobě,,vȯobražati sobě,vȯobražati sobě,?ražati sobě?,,,,,,0


In [868]:
verb_prefixes.query("isv.str.contains('gl.siti')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
2998,17400.0,glåsiti,NaN,v.tr. ipf.,"state, claim, say (text), be worded",NaN,verb,?,?,glåsiti,glåsiti,glås,glåsiti,|glåska|glåsnik,glås,,glås,iti,0
3976,1696.0,izglåsiti,NaN,v.tr. pf.,voice,NaN,verb,?,?,glåsiti,izglåsiti,izglås,glåsiti,,,iz,glås,iti,0
7618,35248.0,ne sȯglasiti,NaN,v.refl. pf.,"disagree, dissent",NaN,verb,?,?,,"ne sȯglasiti sę, ne sųglasiti sę",ne sȯglas,? sȯglasiti?,,,,,,0
9148,173.0,oglåsiti,NaN,v.tr. pf.,"announce, declare, herald",NaN,verb,?,?,glåsiti,oglåsiti,oglås,glåsiti,,,o,glås,iti,0
12008,18514.0,proglåsiti,NaN,v.tr. pf.,proclaim,NaN,verb,?,?,glåsiti,proglåsiti,proglås,glåsiti,,,pro,glås,iti,0
12543,18596.0,råzglåsiti,NaN,v.tr. pf.,"divulge, publicize",NaN,verb,råz’,glåsiti,glåsiti,råzglåsiti,råzglås,glåsiti,,,råz,glås,iti,0
14068,4191.0,sȯglåsiti,NaN,v.refl. ipf.,"agree, consent, concur",NaN,verb,?,?,glåsiti,"sȯglåsiti sę, sųglåsiti sę",sȯglås,glåsiti,,,sȯ,glås,iti,0


In [861]:
verb_prefixes.loc[10615, 'base_verb'] = 'krčiti'

In [799]:
verb_prefixes.query("_stem == '' and _is_irregular == 0").isv.apply(lambda x: bite_all_prefixes_off(x.split(" ")[0], "")[0] in KNOWN_BASES).sum()

13

In [805]:
verb_prefixes.query("_stem == '' and _is_irregular == 0")[
    verb_prefixes.query("_stem == '' and _is_irregular == 0").isv.apply(
        lambda x: bite_all_prefixes_off(
            x.replace("ne ", "").split(" ")[0], ""
        )[0] in KNOWN_BASES
    )
]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
1985,25269.0,doběgati do,(+2),v.intr. ipf.,reach (by running),NaN,verb,?,?,,doběgati do,doběgati do,?běgati do?,,,,,,0
7249,28032.0,naklåti na,(nakolje),v.tr. pf.,pin onto,NaN,verb,?,?,,naklåti na,naklåti na,?klåti na?,,,,,,0
7614,35354.0,ne ljubiti,NaN,v.tr. ipf.,dislike,NaN,verb,?,?,,ne ljubiti,ne ljub,? ljubiti?,,,,,,0
8504,28879.0,obråsti něčim,(obråste),v.intr. pf.,be overgrown with,NaN,verb,?,?,,obråsti něčim,obråsti něčim,?råsti něčim?,,,,,,0
8793,15415.0,odidti od,(odide; odšel),v.tr. pf.,abandon,NaN,verb,?,?,,odidti od,odidti od,?idti od?,,,,,,0
10616,21429.0,pokrčiti ramenami,NaN,v.pf.,shrug,NaN,verb,?,?,,pokrčiti ramenami,pokrčiti ramenami,?krčiti ramenami?,,,,,,0
11116,20055.0,pozrěti (na),(pozri),v.intr. pf.,"look (at), take a look (at), glance (at)",NaN,verb,?,?,,pozrěti (na),pozrěti (na),?zrěti (na)?,,,,,,0
11667,36347.0,pričisliti k,(+3),v.tr. pf.,"number among, rank among",NaN,verb,?,?,,pričisliti k,pričisliti k,?čisliti k?,,,,,,0
12033,11566.0,proizhoditi iz,NaN,v.ipf.,originate from,NaN,verb,?,?,,proizhoditi iz,proizhoditi iz,?hoditi iz?,,,,,,0
12079,32301.0,promyti mozg,(+3),v.tr. pf.,brainwash,NaN,verb,?,?,,promyti mozg,promyti mozg,?myti mozg?,,,,,,0


In [811]:
verb_prefixes.query('isv == "slušati"').base_verb.unique()

array(['slušati'], dtype=object)

In [819]:
candidates = []
for i, row in verb_prefixes.query("_stem == '' and _is_irregular == 0").iterrows():
        x = row.isv.replace("ne ", "").split(" ")[0]
        rest, pref = bite_all_prefixes_off(x, "")
        for rest_cand in [rest, 's' + rest]:
            if rest_cand in KNOWN_BASES:
                lookup = verb_prefixes.query('isv == @rest_cand')
                print(row.isv, '->', rest_cand, '->', lookup.base_verb.unique(), " : ", lookup._prefix.unique(), lookup._stem.unique())
                verb_prefixes.loc[i, "base_verb"] = lookup.base_verb.values[0]


candidates

doběgati do -> běgati -> ['běgati']  :  [''] ['běg']
doběgati do -> sběgati -> ['běgati']  :  ['s'] ['běg']
naklåti na -> klåti -> ['klåti']  :  [''] ['klå']
ne ljubiti -> ljubiti -> ['ljubiti']  :  [''] ['ljub']
ne poslušati -> slušati -> ['slušati']  :  [''] ['sluš']
ne slušati -> slušati -> ['slušati']  :  [''] ['sluš']
obrastati něčim -> srastati -> ['råsti']  :  ['sra'] ['st']
obråsti něčim -> råsti -> ['råsti']  :  ['ra'] ['st']
obråsti něčim -> sråsti -> ['råsti']  :  ['sra'] ['st']
odidti od -> idti -> ['idti']  :  [''] ['jd']
podstųpati k -> stųpati -> ['stųpati']  :  [''] ['stųp']
podstųpiti k -> stųpiti -> ['stųpati']  :  [''] ['stųp']
pokrčiti ramenami -> krčiti -> ['krčiti']  :  [''] ['krč']
pozrěti (na) -> zrěti -> ['zrěti']  :  [''] ['zr']
pričisliti k -> čisliti -> ['čisliti']  :  [''] ['čisl']
pričisliti k -> sčisliti -> ['čisliti']  :  ['s'] ['čisl']
pristųpati do -> stųpati -> ['stųpati']  :  [''] ['stųp']
pristųpati k -> stųpati -> ['stųpati']  :  [''] ['stųp']
pris

[]

In [803]:
verb_prefixes.query("_stem == '' and _is_irregular == 0")[
    verb_prefixes.query("_stem == '' and _is_irregular == 0").isv.apply(lambda x: bite_all_prefixes_off(x.split(" ")[0], "")[0] not in KNOWN_BASES)
]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
1185,22979.0,bųde/bųdųt,NaN,v.aux. ipf.,there will be,NaN,verb,?,?,,bųde/bųdųt,bųde/bųdųt,?bųde/bųdųt?,,,,,,0
2269,35918.0,dovesti do/k,(dovede),v.tr. pf.,lead to,NaN,verb,?,?,,dovesti do/k,dovesti do/k,?esti do/k?,,,,,,0
2275,35917.0,dovoditi do/k,NaN,v.tr. ipf.,lead to,NaN,verb,?,?,,dovoditi do/k,dovoditi do/k,?diti do/k?,,,,,,0
3685,3325.0,iměti dělo s,NaN,v.ipf.,deal with,NaN,verb,?,?,,iměti dělo s,iměti dělo s,?iměti dělo s?,,,,,,0
3686,36890.0,iměti na myslji,NaN,v.tr. ipf.,"mean, have in mind",NaN,verb,?,?,,iměti na myslji,iměti na myslji,?iměti na myslji?,,,,,,0
3687,36928.0,iměti na umu,NaN,v.tr. ipf.,"mean, have in mind",NaN,verb,?,?,,iměti na umu,iměti na umu,?iměti na umu?,,,,,,0
3688,17485.0,iměti nadějų,NaN,v.ipf.,hope,NaN,verb,?,?,,iměti nadějų,iměti nadějų,?iměti nadějų?,,,,,,0
3690,876.0,iměti pravdų,NaN,v.ipf.,be right,NaN,verb,?,?,,iměti pravdų,iměti pravdų,?iměti pravdų?,,,,,,0
3691,36402.0,iměti smysl,NaN,v.ipf.,make sense,NaN,verb,?,?,,iměti smysl,iměti smysl,?iměti smysl?,,,,,,0
3692,23339.0,iměti važnosť,NaN,v.intr. ipf.,matter,NaN,verb,?,?,,iměti važnosť,iměti važnosť,?iměti važnosť?,,,,,,0


In [866]:
verb_prefixes.query("isv.str.contains('stųp')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4283,3505.0,izstųpati,NaN,v.intr. ipf.,"come forward, step forth, appear (on stage etc.)",NaN,verb,?,?,izstųpati,izstųpati,izstųp,stųpati,,,,izstųp,ati,0
4284,21985.0,izstųpati,NaN,v.intr. ipf.,"#withdraw, step out",NaN,verb,?,?,izstųpati,izstųpati,izstųp,stųpati,,,,izstųp,ati,0
4285,3535.0,izstųpiti,NaN,v.intr. pf.,"come forward, step forth, appear (on stage etc.)",NaN,verb,?,?,izstųpati,izstųpiti,izstųp,stųpiti,,,,izstųp,iti,0
4286,21993.0,izstųpiti,NaN,v.intr. pf.,"withdraw, step out",NaN,verb,?,?,izstųpati,izstųpiti,izstųp,stųpiti,,,,izstųp,iti,0
7504,21928.0,nastųpati,NaN,v.intr. ipf.,step on,NaN,verb,?,?,nastųpati,nastųpati,nastųp,stųpati,|nastųpnik|nastųpnica,,,nastųp,ati,0
7505,33299.0,nastųpati,NaN,v.intr. ipf.,"advance, come, set in",NaN,verb,?,?,nastųpati,nastųpati,nastųp,stųpati,|nastųpnik|nastųpnica,,,nastųp,ati,0
7506,21929.0,nastųpiti,NaN,v.intr. pf.,step on,NaN,verb,?,?,nastųpati,nastųpiti,nastųp,stųpiti,|nastųpnik|nastųpnica,,,nastųp,iti,0
7507,28236.0,nastųpiti,NaN,v.intr. pf.,"advance, come, set in",NaN,verb,?,?,nastųpati,nastųpiti,nastųp,stųpiti,|nastųpnik|nastųpnica,,,nastųp,iti,0
8596,21930.0,obstųpati,NaN,v.tr. ipf.,surround,NaN,verb,?,?,obstųpati,obstųpati,obstųp,stųpati,,,,obstųp,ati,0
8597,21931.0,obstųpiti,NaN,v.tr. pf.,surround,NaN,verb,?,?,obstųpati,obstųpiti,obstųp,stųpiti,,,,obstųp,iti,0


In [867]:
verb_prefixes.query("isv.str.contains('stųp')")

verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('stųp')").index,
    "base_verb"
] = 'stųpati'

In [863]:
verb_prefixes.loc[
    verb_prefixes.query("isv.str.contains('vlěk')").index,
    "base_verb"
] = 'vlěkti'

In [678]:
IDX = verb_prefixes.query('(base_verb == "ti") and isv.str.contains("sti")').index

verb_prefixes.loc[IDX, 'base_verb'] = 'nositi'



In [755]:
IDX = verb_prefixes.query('(base_verb == "kajati" or base_verb == "pokojiti") or en.str.contains("nottt")').index


verb_prefixes.loc[IDX, "derived_nouns"] = 'cěna'


In [765]:
IDX = verb_prefixes.query('base_verb == "izvlastniti" or base_noun == "vlasť"').index
IDX = verb_prefixes.query('isv.str.contains("vlas")').index

verb_prefixes.loc[IDX, 'base_verb'] = "vlastniti"

manual_insert("vlastniti", "vlast", {}, dry_run=False)


izvlastniti = iz vlast niti
izvlastnjati = iz vlast njati
privlastniti sobě = pri vlast niti
privlastnjati sobě = pri vlast njati
vlastniti =  vlast niti


In [764]:
IDX = verb_prefixes.query('isv.str.contains("vlas")').index
verb_prefixes.loc[IDX]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4359,22512.0,izvlastniti,NaN,v.tr. pf.,expropriate,NaN,verb,?,?,vlastniti,izvlastniti,izvlastn,vlastniti,,,iz,vlast,niti,0
4360,22511.0,izvlastnjati,NaN,v.tr. ipf.,expropriate,NaN,verb,?,?,vlastniti,izvlastnjati,izvlastn,?lastnjati?,,,iz,vlast,njati,0
11911,22501.0,privlastniti sobě,NaN,v.tr. pf.,"appropriate, arrogate",NaN,verb,?,?,,privlastniti sobě,privlastniti sobě,?lastniti sobě?,,,,,,0
11912,22502.0,privlastnjati sobě,NaN,v.tr. ipf.,"appropriate, arrogate",NaN,verb,?,?,,privlastnjati sobě,privlastnjati sobě,?lastnjati sobě?,,,,,,0
16511,13681.0,vlastniti,NaN,v.tr. ipf.,"own, possess",NaN,verb,?,?,vlastniti,vlastniti,vlastn,?lastniti?,|vlastnica|vlastnosť,vlasť,,vlast,niti,0


In [ ]:
verb_prefixes.query('(base_verb == "kajati" or base_verb == "pokojiti") or en.str.contains("nottt")')

In [752]:
verb_prefixes.query('(base_verb == "†pokojiti")')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
784,36698.0,bezpokojiti,NaN,v.refl. ipf.,"worry, be anxious, feel uneasy",NaN,verb,?,?,†pokojiti,bezpokojiti sę,bezpoko,?kojiti?,,,bez,poko,jiti,0
785,19654.0,bezpokojiti,NaN,v.tr. ipf.,"disturb, trouble, upset, bother",NaN,verb,?,?,†pokojiti,bezpokojiti,bezpoko,?kojiti?,,,bez,poko,jiti,0
7844,36699.0,nepokojiti,NaN,v.refl. ipf.,"worry, be anxious, feel uneasy",NaN,verb,?,?,†pokojiti,nepokojiti sę,nepoko,?kojiti?,,,ne,poko,jiti,0
7845,3862.0,nepokojiti,NaN,v.tr. ipf.,worry,NaN,verb,?,?,†pokojiti,nepokojiti,nepoko,?kojiti?,,,ne,poko,jiti,0
8302,19655.0,obezpokojiti,NaN,v.tr. pf.,"disturb, trouble, upset, bother",NaN,verb,obez’,pokojiti,†pokojiti,obezpokojiti,obezpoko,bezpokojiti,,,obez,poko,jiti,0
15874,1590.0,uspokajati,NaN,v.tr. ipf.,"calm, appease, pacify, sooth, allay",NaN,verb,?,?,†pokojiti,uspokajati,uspoka,kajati,,,,,,0
15877,1589.0,uspokojiti,NaN,v.tr. pf.,"calm, appease, pacify, sooth, allay",NaN,verb,?,?,†pokojiti,uspokojiti,uspoko,?kojiti?,,,us,poko,jiti,0
17336,35852.0,zanepokojiti,NaN,v.tr. pf.,worry,NaN,verb,?,?,†pokojiti,zanepokojiti,zanepoko,nepokojiti,,,zane,poko,jiti,0


In [753]:
verb_prefixes.loc[15607, 'base_verb'] = 'daviti'
verb_prefixes.loc[15874, 'base_verb'] = '†pokojiti'

verb_prefixes.loc[15874,  ['_prefix', '_stem', '_suffix']] = 'po', 'k' + AO + 'j', 'iti'


In [685]:
IDX = verb_prefixes.query('(base_verb == "ti") and en.str.contains("howl")').index

verb_prefixes.loc[IDX, 'base_verb'] = 'vyti'

IDX = verb_prefixes.query('(base_verb == "ti") and en.str.contains("shoes")').index
verb_prefixes.loc[IDX, 'base_verb'] = 'obuti'


In [689]:
IDX = verb_prefixes.query('(base_verb == "ti") ').index

verb_prefixes.loc[IDX, 'base_verb'] = 'poriti'



In [688]:
verb_prefixes.query('base_verb.str.contains("por")')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
1806,398.0,deportovati,NaN,v.tr. ipf./pf.,deport,I,verb,de’,portovati,deportovati,deportovati,deport,?rtovati?,|deportacija,,,deporto,vati,0
2573,371.0,eksportovati,NaN,v.tr. ipf.,export,I,verb,?,?,eksportovati,eksportovati,eksport,eksportovati,,eksport,,eksporto,vati,0
3711,4962.0,importovati,NaN,v.tr. ipf.,import,I,verb,?,?,importovati,importovati,import,importovati,,import,,importo,vati,0
9399,20001.0,oporędčati,NaN,v.tr. ipf.,order,NaN,verb,?,?,oporędčati,oporędčati,oporędč,?rędčati?,,,,oporędč,ati,0
9401,20003.0,oporędčiti,NaN,v.tr. pf.,order,NaN,verb,?,?,oporędčati,oporędčiti,oporędč,?rędčiti?,,,,oporędč,iti,0
9644,5910.0,osparjati,NaN,v.tr. ipf.,"dispute, contest, question",NaN,verb,?,?,sporiti,osparjati,ospar,?parjati?,,,o,spꜵr,jati,0
9645,5939.0,osporiti,NaN,v.tr. pf.,"dispute, contest, question",NaN,verb,?,?,sporiti,osporiti,ospor,riti,,,o,spꜵr,iti,0
10444,10758.0,podporiti,NaN,v.tr. pf.,"support, back, back up",NaN,verb,?,?,poriti,podporiti,podpor,riti,|podporka|podpornik,,,podpor,iti,0
12779,23927.0,råzporęditi,NaN,v.tr. pf.,"arrange, dispose",NaN,verb,råz’,poręditi,råzporęditi,råzporęditi,råzporęd,?ręditi?,,,,råzporęd,iti,0
12780,23929.0,råzporęđati,NaN,v.tr. ipf.,"arrange, dispose",NaN,verb,råz’,poręđati,råzporęditi,råzporęđati,råzporęđ,?ręđati?,,,,råzporęd,յati,0


In [670]:
#    if k == 'pnųti':
#        true_stem = "pn"
IDX = verb_prefixes.query('(base_verb == "vajati" or base_verb == "svojiti") and not en.str.contains("sculpt")').index

for i in IDX:
    word = verb_prefixes.loc[[i], 'isv'].values[0]
    verb_prefixes.loc[[i], 'base_verb'] = 'svojiti'
    verb_prefixes.loc[[i], 'base_noun'] = 'svoj'
    stem = "svoj" if "svoj" in word else "svaj"
    verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix']] = word.partition(stem)

# dva svoj vajati

'osvajati'

In [663]:
verb_prefixes.query('base_verb == "dvojiti"')
#podvajati


verb_prefixes.loc[[10492], 'base_verb'] = 'dvojiti'
verb_prefixes.loc[[10492], '_prefix'] = 'po'
verb_prefixes.loc[[10492], '_stem'] = 'dvaj'
verb_prefixes.loc[[10492], '_suffix'] = 'ati'


In [657]:
#    if k == 'pnųti':
#        true_stem = "pn"
verb_prefixes.query('base_verb == "tajati"')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
8590,9895.0,obstajati,NaN,v.intr. ipf.,exist,NaN,verb,?,?,tajati,obstajati,obsta,tajati,,,,,,0
8591,9897.0,obstajati,NaN,v.intr. ipf.,"stay, spend time",NaN,verb,?,?,tajati,obstajati,obsta,tajati,,,,,,0
9070,29274.0,odtajati,NaN,v.tr. pf.,"thaw, unfreeze",NaN,verb,?,?,tajati,odtajati,odta,tajati,,,,,,0
12918,30522.0,råztajati,NaN,v.tr. pf.,"melt, thaw",NaN,verb,råz’,tajati,tajati,råztajati,råzta,tajati,,,,,,0
15018,34023.0,tajati,NaN,v.tr. ipf.,"melt, thaw, unfreeze",NaN,verb,?,?,tajati,tajati,ta,tajati,,,,,,0


In [658]:
verb_prefixes.loc[[8590, 8591], 'base_verb'] = 'stojati'
verb_prefixes.loc[[8590, 8591], '_stem'] = 'staj'
verb_prefixes.loc[[8590, 8591], '_suffix'] = 'ati'


In [651]:
verb_prefixes.query('en.str.contains("stretch") and isv.str.contains("pi?n")')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
2321,442.0,drapnųti,NaN,v.tr. pf.,scratch,NaN,verb,?,?,drapati,drapnųti,drap,drapnųti,,,,drap,nųti,0
3424,19715.0,hipnotizovati,NaN,v.tr. ipf.,hypnotize,I,verb,?,?,hipnotizovati,hipnotizovati,hipnotiz,hipnotizovati,,,,hipnotizo,vati,0
3454,17458.0,hlipnųti,NaN,v.intr. pf.,sob,NaN,verb,?,?,hlipati,hlipnųti,hlip,hlipnųti,,,,hlip,nųti,0
3512,17467.0,hrapnųti,NaN,v.intr. pf.,snore,NaN,verb,?,?,hrapati,hrapnųti,hrap,hrapnųti,,,,hrap,nųti,0
4790,26470.0,kapnųti,NaN,v.intr. pf.,"drip, drop",NaN,verb,?,?,kapati,kapnųti,kap,kapnųti,,,,kap,nųti,0
7328,28098.0,napinati,NaN,v.tr. ipf.,"stretch, strain, pull tight",NaN,verb,?,?,pinati,napinati,napin,pinati,,,na,pin,ati,0
8916,29129.0,odpinati,NaN,v.tr. ipf.,"unbutton, unclasp, unfasten",NaN,verb,?,?,pinati,odpinati,odpin,pinati,,,od,pin,ati,0
9250,29398.0,okrěpnųti,NaN,v.intr. pf.,become stronger,NaN,verb,?,?,krěpiti,okrěpnųti,okrěp,?krěpnųti?,,,o,krěp,nųti,0
9583,29639.0,oslěpnųti,NaN,v.intr. pf.,go blind,NaN,verb,?,?,slěpnųti,oslěpnųti,oslěp,slěpnųti,,,o,slěp,nųti,0
10148,20990.0,pinati,NaN,v.tr. ipf.,"stretch, pull tight",NaN,verb,?,?,pinati,pinati,pin,pinati,,,,pin,ati,0


In [652]:
verb_prefixes.loc[10268, 'base_verb'] = 'pinati'
verb_prefixes.loc[10268, '_stem'] = 'pn'
verb_prefixes.loc[10268, '_suffix'] = 'ųti'
verb_prefixes.loc[10268, '_is_irregular'] = 1


In [642]:
LSC = ['dozvꜵl', 'dozvꜵl', 'izvꜵljn', 'izvꜵl', 'izvꜵljn', 'nezadovꜵl', 'nezadovꜵl', 'pozvꜵl', 'pozvꜵl', 'pozvꜵl', 'pozvꜵl', 'privꜵl', 'privꜵl', 'uvꜵljn', 'uvꜵljn', 'uvꜵljn', 'uvꜵljn', 'uvꜵljn', 'uvꜵljn', 'uvꜵljn', 'uvꜵljn', 'zadovꜵl', 'zadovꜵl', 'zadovꜵl', 'zadovꜵl']

LSC = [l.replace("jn", "") for l in LSC]
common_suffix(LSC)


'vꜵl'

In [638]:
verb_prefixes.query('base_verb == "voliti"')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
2294,435.0,dozvaljati,NaN,v.tr. ipf.,"let, permit, allow",NaN,verb,?,?,voliti,dozvaljati,dozval,?zvaljati?,,,,,,0
2296,414.0,dozvoliti,NaN,v.tr. pf.,"let, permit, allow",NaN,verb,?,?,voliti,dozvoliti,dozvol,?zvoliti?,,,,,,0
4336,763.0,izvaljnjati,NaN,v.tr. ipf.,"fire (terminate employment), sack, discharge, ...",NaN,verb,?,?,voliti,izvaljnjati,izvaljn,?aljnjati?,,,,,,0
4374,14611.0,izvoliti,NaN,v.tr. pf.,elect,NaN,verb,?,?,voliti,izvoliti,izvol,liti,,,,,,0
4377,2956.0,izvoljniti,NaN,v.tr. pf.,"fire (terminate employment), sack, discharge, ...",NaN,verb,?,?,voliti,izvoljniti,izvoljn,?ljniti?,,,,,,0
8064,17948.0,nezadovaljati,NaN,v.tr. ipf.,"displease, dissatisfy",NaN,verb,?,?,voliti,nezadovaljati,nezadoval,valjati,,,,,,0
8065,17953.0,nezadovoliti,NaN,v.tr. pf.,"displease, dissatisfy",NaN,verb,?,?,voliti,nezadovoliti,nezadovol,liti,,,,,,0
11118,11124.0,pozvaljati sobě,NaN,v.ipf.,afford,NaN,verb,?,?,voliti,pozvaljati sobě,pozvaljati sobě,?zvaljati sobě?,,,,,,0
11119,3156.0,pozvaljati,NaN,v.tr. ipf.,"allow, let, permit",NaN,verb,?,?,voliti,pozvaljati,pozval,?zvaljati?,,,,,,0
11125,11120.0,pozvoliti sobě,NaN,v.pf.,afford,NaN,verb,?,?,voliti,pozvoliti sobě,pozvoliti sobě,?zvoliti sobě?,,,,,,0


In [635]:
verb_prefixes.query('_stem == "" and base_noun == "" and base_verb != ""')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
1704,25137.0,daviti,NaN,v.tr. ipf.,strangle,NaN,verb,?,?,daviti,daviti,dav,daviti,|davec|davač,,,,,0
1751,6992.0,dějati,(děje),v.refl. ipf.,happen,NaN,verb,?,?,dějati,dějati sę,dě,dějati,,,,,,0
1752,6991.0,dějati,(děje),v.tr. ipf.,act,NaN,verb,?,?,dějati,dějati,dě,dějati,,,,,,0
2154,3813.0,donesti,NaN,v.tr. pf.,"report, inform",NaN,verb,?,?,ti,donesti,donesti,ti,,,,,,0
2294,435.0,dozvaljati,NaN,v.tr. ipf.,"let, permit, allow",NaN,verb,?,?,voliti,dozvaljati,dozval,?zvaljati?,,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17133,444.0,zadovaljati,NaN,v.tr. ipf.,"satisfy, please",NaN,verb,?,?,voliti,zadovaljati,zadoval,valjati,,,,,,0
17134,36517.0,zadovoliti,NaN,v.refl. pf.,"be content with, be satisfied with, to get by ...",NaN,verb,?,?,voliti,zadovoliti sę,zadovol,liti,,,,,,0
17135,3356.0,zadovoliti,NaN,v.tr. pf.,"satisfy, please",NaN,verb,?,?,voliti,zadovoliti,zadovol,liti,,,,,,0
17573,33351.0,zavyti,NaN,v.intr. pf.,"howl, wail",NaN,verb,?,?,ti,zavyti,zavyti,ti,,,,,,0


In [603]:
IDX = verb_prefixes.query('_is_irregular == -1 and _stem == ""').index

bad_stems = verb_prefixes.loc[IDX].base_verb.unique()

IDX = verb_prefixes.query('base_verb in @bad_stems').index
for i in IDX:
    verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["", "", "", 0]


In [613]:
tmp_df = verb_prefixes.query(" en.str.contains('dry') and  (isv.str.contains('suš') or isv.str.contains('sȯh')  or isv.str.contains('syh'))")

for i, row in tmp_df.iterrows():
    if row['_stem'][-1:] == 'š':
        pref, suf = row.isv.split('suš')
        verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix', '_is_irregular']] = [pref, "suh", YI + suf, 0]


IDX = verb_prefixes.query("base_verb.str.contains('nazdråviti')").index

#nazdråviti
for i in IDX:
    a = verb_prefixes.loc[i, "isv"][3:7]
    b = verb_prefixes.loc[i, "isv"][7:]
    verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["naz", a, b, 0]
verb_prefixes.query("base_verb.str.contains('nazdråviti')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
7594,32964.0,nazdråviti,NaN,v.intr. pf.,"toast, drink a toast",NaN,verb,?,?,nazdråviti,nazdråviti,nazdråv,?zdråviti?,,,naz,dråv,iti,0
7595,32962.0,nazdravjati,NaN,v.intr. ipf.,"toast, drink a toast",NaN,verb,?,?,nazdråviti,nazdravjati,nazdrav,?zdravjati?,,,naz,drav,jati,0


In [600]:
verb_prefixes.loc[12632, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["råz", "krųt", "յati", 1]


IDX = verb_prefixes.query("base_verb == 'klęti' and _suffix.str.contains('ę')").index
for i in IDX:
    verb_prefixes.loc[i, ['_stem', '_suffix', '_is_irregular']] = ["klę", "ti", 0]

verb_prefixes.loc[17230, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["za", "klin", "ati", 1]

verb_prefixes.loc[12534, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["råz", "dvaj", "ati", 1]

verb_prefixes.loc[8366, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["ob", "krad", "ati", 1]



In [575]:
verb_prefixes.query("isv.str.contains('povŕt')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
11065,22659.0,povŕtati,NaN,v.tr. ipf.,turn (tr.),NaN,verb,?,?,vŕnųti,povŕtati,povŕt,?ŕtati?,,,,,,0.0


In [574]:
[bite_all_suffixes_off(insert_YI("začęti", {'č', 'n'}), ""), 

bite_all_suffixes_off(insert_YI("začinati", {'č', 'n'}), "")
]

[('začin', '˛+ti+'), ('začin', 'a+ti+')]

In [557]:
signatures_cnt

Counter({'č_n': 2,
         'k_č': 2,
         'ć_č': 1,
         'š_s': 1,
         't_s': 1,
         'ž_g': 1,
         't_ć': 1,
         'r_v': 1,
         'z__s_d_v': 1,
         'l_n': 1,
         't_ŕ': 1,
         'n_g': 1})

In [605]:
verb_prefixes._prefix.value_counts()

          3145
iz         159
od         148
o          130
za         124
          ... 
izȯ          1
spolu        1
izu          1
su           1
nedora       1
Name: _prefix, Length: 107, dtype: int64

In [606]:
verb_prefixes._suffix.value_counts()

                      2076
iti                    820
ati                    744
jati                   273
vati                   207
ti                     198
յati                   152
nųti                   152
ěti                    100
ivati                   50
avjati                  35
aviti                   32
ěvati                   19
vyvati                  16
jiti                    13
yvati                   10
vti                      9
oviti                    9
stti                     5
յiti                     4
avati                    4
tati                     3
ějati                    3
ějivati                  3
˛ti                      3
յivati                   3
in˛ti                    3
omiti                    2
ati v sųd                2
ipraviti                 2
ipravjati                2
inųti                    2
amjati                   2
ajati                    2
ajivati                  2
vati roljų               1
vati šnurom              1
a

In [607]:
IDX = verb_prefixes.query('_suffix.str.contains(" ")').index

for i in IDX:
    suf = verb_prefixes.loc[i, '_suffix']
    verb_prefixes.loc[i, '_suffix'] = suf.split(" ")[0]


In [608]:
verb_prefixes.query('_stem == "" and genesis == genesis')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
5,11.0,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija,,,,,0
16,6119.0,abonovati,NaN,v.tr. ipf.,"subscribe, engage",F,verb,?,?,abonovati,abonovati,abon,abonovati,,,,,,0
28,19625.0,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,,,,,0
29,24056.0,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,,,,,0
43,24070.0,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17406,20833.0,zaprojektovati,NaN,v.tr. pf.,design,I,verb,?,?,projektovati,zaprojektovati,zaprojekt,projektovati,,,,,,0
17425,25.0,zaregistrovati,NaN,v.tr. pf.,register,I,verb,?,?,registrovati,zaregistrovati,zaregistr,registrovati,,,,,,0
17429,35859.0,zarezervovati,NaN,v.tr. pf.,"reserve, book",I,verb,?,?,rezervovati,zarezervovati,zarezerv,rezervovati,,,,,,0
17501,5616.0,zaštopati,NaN,v.tr. pf.,darn,I,verb,?,?,štopati,zaštopati,zaštop,štopati,,,,,,0


In [609]:
vc = verb_prefixes.query('_stem == "" and genesis != genesis and base_noun == ""').base_verb.value_counts()

vc[vc > 1]

             71
ti           30
voliti       26
hoditi       25
staviti      23
             ..
zlobiti       2
smatrjati     2
kašljati      2
sŕditi        2
klåtiti       2
Name: base_verb, Length: 277, dtype: int64

In [616]:
verb_prefixes.query('_stem == "" and genesis != genesis and base_noun == ""').base_verb.value_counts()

             71
ti           30
voliti       26
hoditi       25
staviti      23
             ..
lojiti        1
loskotati     1
malěti        1
mekati        1
žužati        1
Name: base_verb, Length: 479, dtype: int64

In [611]:
verb_prefixes.query('_stem == "" and genesis != genesis and base_noun != ""')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
632,5873.0,barviti,NaN,v.tr. ipf.,"paint, colour, color",NaN,verb,?,?,barviti,barviti,barv,barviti,,barva,,,,0
679,2441.0,běgati,NaN,v.intr. ipf.,run,NaN,verb,?,?,běgati,běgati,běg,běgati,|běgstvo,běg,,,,0
712,2880.0,besědovati,NaN,v.intr. ipf.,"converse, chat",NaN,verb,?,?,besědovati,besědovati,besěd,besědovati,,besěda,,,,0
713,24612.0,běsiti,NaN,v.refl. ipf.,rage,NaN,verb,?,?,běsiti,běsiti sę,běs,běsiti,,běs,,,,0
920,24688.0,blazniti,NaN,v.refl. ipf.,behave like a fool,NaN,verb,?,?,blazniti,blazniti sę,blazn,blazniti,,blazn,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17756,34748.0,zvěrěti,NaN,v.intr. ipf.,"go wild, become brutalized",NaN,verb,?,?,zvěrěti,zvěrěti,zvěr,zvěrěti,,!zvěŕ,,,,0
17768,3822.0,zvoniti,NaN,v.intr. ipf.,"ring, chime",NaN,verb,?,?,zvoniti,zvoniti,zvon,zvoniti,|zvonica,zvon,,,,0
17792,19402.0,žariti,NaN,v.tr. ipf.,"incandesce, heat to a glow",NaN,verb,?,?,žariti,žariti,žar,žariti,,žar,,,,0
17827,3399.0,ženiti,NaN,v.refl. ipf.,"marry (with a woman), get married (with a woman)",NaN,verb,?,?,ženiti,ženiti sę,žen,ženiti,,žena,,,,0


In [614]:

verb_prefixes.query("base_verb == 'šiti' and en.str.contains('dry')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
4290,33097.0,izsušati,NaN,v.tr. ipf.,"dehydrate, dry out",NaN,verb,?,?,šiti,izsušati,izsuš,?šati?,,,iz,suh,յati,0
4291,33098.0,izsušiti,NaN,v.tr. pf.,"dehydrate, dry out",NaN,verb,?,?,šiti,izsušiti,izsuš,šiti,,,iz,suh,յiti,0
9691,29718.0,osušati,NaN,v.tr. ipf.,"dry, drain",NaN,verb,?,?,šiti,osušati,osuš,?šati?,,,o,suh,յati,0
9692,29719.0,osušiti,NaN,v.tr. pf.,"dry, drain",NaN,verb,?,?,šiti,osušiti,osuš,šiti,,,o,suh,յiti,0
14699,18925.0,sušiti,NaN,v.tr. ipf.,dry,NaN,verb,?,?,šiti,sušiti,suš,šiti,,,,suh,յiti,0


In [527]:
verb_prefixes.query("base_verb == 'råsti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
2179,7211.0,dorastati,NaN,v.intr. ipf.,"grow up, become an adult",NaN,verb,?,?,råsti,dorastati,dorast,?rastati?,,,dorastati,,,-1.0
2180,16416.0,doråsti,(doråste),v.intr. pf.,"grow up, become an adult",NaN,verb,?,?,råsti,doråsti,doråsti,råsti,,,do,rå,sti,-1.0
4220,1688.0,izråsti,(izråste),v.intr. pf.,grow,NaN,verb,?,?,råsti,izråsti,izråsti,råsti,,,iz,rå,sti,-1.0
7378,33056.0,narastati,NaN,v.intr. ipf.,"increase, rise, grow, accrue",NaN,verb,?,?,råsti,narastati,narast,?rastati?,,,narastati,,,-1.0
7379,28139.0,naråsti,(naråste),v.intr. pf.,"increase, rise, grow, accrue",NaN,verb,?,?,råsti,naråsti,naråsti,råsti,,,na,rå,sti,-1.0
10816,23903.0,poråsti,(poråste),v.intr. pf.,grow,NaN,verb,?,?,råsti,poråsti,poråsti,råsti,,,po,rå,sti,-1.0
11492,33068.0,prěrastati,NaN,v.tr. ipf.,outgrow,NaN,verb,?,?,råsti,prěrastati,prěrast,?rastati?,,,prěrastati,,,-1.0
11493,11338.0,prěråsti,(prěråste),v.tr. pf.,outgrow,NaN,verb,?,?,råsti,prěråsti,prěråsti,råsti,,,prě,rå,sti,-1.0
12433,3037.0,råsti,(råste),v.intr. ipf.,"grow, increase",NaN,verb,?,?,råsti,råsti,råsti,råsti,,,,rå,sti,-1.0
12825,30427.0,råzrastati,NaN,v.refl. ipf.,"branch out, expand, run wild",NaN,verb,råz’,rastati sę,råsti,råzrastati sę,råzrast,?rastati?,,,,rå,zrastati,-1.0


In [508]:
verb_prefixes.drop(
    [x for x in verb_prefixes.index if x == str(x)], inplace=True
)

In [500]:
(5004, 2942)


(5004, 2942)

In [502]:
IDX = verb_prefixes.query('_is_irregular == -1').index

for i in IDX:
    verb_prefixes.loc[i, ['_prefix', '_stem', '_suffix', '_is_irregular']] = ["", "", "", 0]
IDX

Index([  654,  1101,  1700,  1701,  1702,  1836,  1837,  1983,  1984,  2036,
       ...
       17417, 17418, 17500, 17545, 17564, 17565, 17623, 17624, 17845, 17846],
      dtype='object', length=361)

In [495]:
verb_prefixes.query('_is_irregular == -1').sort_values(by='_stem')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
10551,4309.0,pogynųti,NaN,v.intr. pf.,perish,NaN,verb,?,?,†gynųti,pogynųti,pogy,?gynųti?,,,pogynųti,,,-1.0
11118,11124.0,pozvaljati sobě,NaN,v.ipf.,afford,NaN,verb,?,?,voliti,pozvaljati sobě,pozvaljati sobě,?zvaljati sobě?,,,pozvaljati sobě,,,-1.0
11065,22659.0,povŕtati,NaN,v.tr. ipf.,turn (tr.),NaN,verb,?,?,vŕnųti,povŕtati,povŕt,?ŕtati?,,,povŕtati,,,-1.0
10550,4308.0,pogynati,NaN,v.intr. ipf.,perish,NaN,verb,?,?,†gynųti,pogynati,pogyn,?gynati?,,,pogynati,,,-1.0
4189,22798.0,izprašati,NaN,v.tr. ipf.,beg,NaN,verb,?,?,prositi,izprašati,izpraš,?prašati?,,,izprasյati,,,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16030,22932.0,užiti,(užive),v.tr. pf.,"use, enjoy",NaN,verb,?,?,žiti,užiti,už,žiti,,,u,ž,iti,-1.0
16032,22931.0,uživati,NaN,v.tr. ipf.,"use, enjoy",NaN,verb,?,?,žiti,uživati,už,?živati?,,,u,ž,ivati,-1.0
9843,29815.0,ožiti,(ožive),v.intr. pf.,come to life,NaN,verb,?,?,žiti,ožiti,ož,žiti,,,o,ž,iti,-1.0
17053,3673.0,vživati,NaN,v.refl. ipf.,"get used (to), get accustomed (to), familiariz...",NaN,verb,?,?,žiti,vživati sę,vž,?živati?,,,v,ž,ivati,-1.0


In [496]:
verb_prefixes.query('_is_irregular == -1')['_stem'].value_counts()

           60
jav        26
d          22
p          18
ply        17
ž          17
dav        17
v          15
vol        14
š          12
rå         11
råbot      11
b          11
trat       10
vlě         8
ple         8
věd         7
pros        6
vaj         6
krųt        5
laj         5
kl          5
kra         5
oblě        4
dvoj        4
služ        4
vŕn         3
brån        3
obč         3
poj         3
način       2
sȯs         2
spor        2
nav         2
ověj        2
bran        2
zavojev     2
zěv         2
rån         1
ran         1
nazdråv     1
uzdråv      1
začin       1
Name: _stem, dtype: int64

In [475]:
 
verb_prefixes.query("base_verb.str.contains('měšati')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,bas_verb
6549,2839,měšati,NaN,v.tr. ipf.,"mix, blend",NaN,verb,?,?,měšati,měšati,měš,měšati,,,,,,0,NaN
12697,1041,råzměšati,NaN,v.tr. pf.,mix up,NaN,verb,råz’,měšati,měšati,råzměšati,råzměš,měšati,,,,,,0,NaN
12698,12376,råzměšati,NaN,v.tr. pf.,puzzle,NaN,verb,råz’,měšati,měšati,råzměšati,råzměš,měšati,,,,,,0,NaN
12701,3004,råzměšivati,NaN,v.tr. ipf.,mix up,NaN,verb,råz’,měšivati,měšati,råzměšivati,råzměš,?měšivati?,,,,,,0,NaN
12702,16898,råzměšivati,NaN,v.tr. ipf.,puzzle,NaN,verb,råz’,měšivati,měšati,råzměšivati,råzměš,?měšivati?,,,,,,0,NaN
12869,23930,råzsměšati,NaN,v.tr. ipf.,make laugh,NaN,verb,råz’,směšati,měšati,råzsměšati,råzsměš,měšati,,,,,,0,NaN
12870,23931,råzsměšiti,NaN,v.tr. pf.,make laugh,NaN,verb,råz’,směšiti,měšati,råzsměšiti,råzsměš,?měšiti?,,,,,,0,NaN
13956,1817,směšati,NaN,v.tr. pf.,"mix, blend",NaN,verb,?,?,měšati,směšati,směš,měšati,|směška,,,,,0,NaN
13957,14274,směšati,NaN,v.tr. pf.,perplex,NaN,verb,?,?,měšati,směšati,směš,měšati,|směška,,,,,0,NaN
13958,18845,směšivati,NaN,v.tr. ipf.,"mix, blend",NaN,verb,?,?,měšati,směšivati,směš,?měšivati?,|směška,,,,,0,NaN


In [460]:
signatures_cnt

Counter({'đ_d': 33,
         'd_v': 2,
         'š_s': 12,
         'z_ž': 6,
         'v_n': 3,
         'b_v': 1,
         't_ć': 26,
         'z__r_d_b_p': 1,
         'đ_h': 1,
         'l_n': 2,
         'k_v': 2,
         'd_s': 1,
         'r_v': 5,
         'č_n': 2,
         'k_č': 2,
         'ć_č': 1,
         't_s': 2,
         'l_v': 1,
         'ž_g': 1,
         'z__s_d_v': 1,
         '_v': 1,
         'd_č_m': 1,
         't_ŕ': 1,
         'n_g': 1})

In [471]:
verb_prefixes.query("isv.str.contains('idti') or isv.str.contains('imati')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,bas_verb
130,24156,aklimatizovati,NaN,v.intr. ipf.,acclimatize,I,verb,?,?,aklimatizovati,aklimatizovati,aklimatiz,aklimatizovati,|aklimatizacija,,,,,0,NaN
3638,13823,idti vprěd,#(ide; šel),v.ipf.,advance,NaN,verb,?,?,idti,idti vprěd,idti vprěd,?idti vprěd?,,,,,,0,NaN
3639,15654,idti vzad,#(ide; šel),v.ipf.,back up,NaN,verb,?,?,idti,idti vzad,idti vzad,?idti vzad?,,,,,,0,NaN
3640,2104,idti,(ide; šel),v.intr. ipf.,go,NaN,verb,?,?,idti,idti,idti,idti,,,,,,0,NaN
3695,417,"iměti, imati",(imaje),v.aux. ipf.,"must, have to",NaN,verb,?,?,,"iměti, imati","iměti, im","?iměti, imati?",,,,,,0,NaN
3696,875,"iměti, imati",(imaje),v.tr. ipf.,"have, possess, own",NaN,verb,?,?,,"iměti, imati","iměti, im","?iměti, imati?",,,,,,0,NaN
4011,5515,izimati,NaN,v.tr. ipf.,take out,NaN,verb,?,?,imati,izimati,izim,imati,,,,,,0,NaN
8329,3476,obimati,NaN,v.tr. ipf.,"embrace, hug, clasp",NaN,verb,?,?,imati,obimati,obim,imati,,,,,,0,NaN
8793,15415,odidti od,(odide; odšel),v.tr. pf.,abandon,NaN,verb,?,?,,odidti od,odidti od,?idti od?,,,,,,0,NaN
8798,18066,odimati,NaN,v.tr. ipf.,take away,NaN,verb,?,?,imati,odimati,odim,imati,,,,,,0,NaN


In [470]:
(4985, 2653)
verb_prefixes.query("base_verb == 'idti' or base_verb == 'imati'")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,bas_verb
2057,5345,dojdti,(dojde; došel),v.intr. pf.,arrive,NaN,verb,?,?,idti,dojdti,dojdti,?jdti?,,,,,,0,NaN
3638,13823,idti vprěd,#(ide; šel),v.ipf.,advance,NaN,verb,?,?,idti,idti vprěd,idti vprěd,?idti vprěd?,,,,,,0,NaN
3639,15654,idti vzad,#(ide; šel),v.ipf.,back up,NaN,verb,?,?,idti,idti vzad,idti vzad,?idti vzad?,,,,,,0,NaN
3640,2104,idti,(ide; šel),v.intr. ipf.,go,NaN,verb,?,?,idti,idti,idti,idti,,,,,,0,NaN
4011,5515,izimati,NaN,v.tr. ipf.,take out,NaN,verb,?,?,imati,izimati,izim,imati,,,,,,0,NaN
4148,5354,izȯjdti,(izȯjde; izšel),v.intr. pf.,"exit, go out, quit",NaN,verb,?,?,idti,izȯjdti,izȯjdti,?ȯjdti?,,,,,,0,NaN
7187,781,najdti,(najde; našel),v.tr. pf.,find,NaN,verb,?,?,idti,najdti,najdti,?dti?,,,,,,0,NaN
8329,3476,obimati,NaN,v.tr. ipf.,"embrace, hug, clasp",NaN,verb,?,?,imati,obimati,obim,imati,,,,,,0,NaN
8469,18025,obȯjdti,(obȯjde; obšel),v.refl. pf.,do without,NaN,verb,?,?,idti,obȯjdti sę bez,obȯjdti,?ȯjdti?,,,,,,0,NaN
8470,18023,obȯjdti,(obȯjde; obšel),v.refl. pf.,"manage, make do with, get by",NaN,verb,?,?,idti,obȯjdti sę,obȯjdti,?ȯjdti?,,,,,,0,NaN


In [453]:
# verb_prefixes.query("en.str.contains('agree')")
# verb_prefixes.query("base_verb == 'voliti'")

verb_prefixes.query("isv.str.contains('val')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,bas_verb
1944,35295,diskvalifikovati,NaN,v.tr. ipf./pf.,disqualify,I,verb,?,?,diskvalifikovati,diskvalifikovati,diskvalifik,diskvalifikovati,|diskvalifikacija,,,,,0,NaN
2294,435,dozvaljati,NaN,v.tr. ipf.,"let, permit, allow",NaN,verb,?,?,dozvoliti,dozvaljati,dozval,?zvaljati?,,,,dozvꜵl,jati,0,NaN
3574,2890,hvaliti,NaN,v.tr. ipf.,praise,NaN,verb,?,?,hvaliti,hvaliti,hval,hvaliti,,hvala,,,,0,NaN
4336,763,izvaljnjati,NaN,v.tr. ipf.,"fire (terminate employment), sack, discharge, ...",NaN,verb,?,?,izvoljniti,izvaljnjati,izvaljn,?aljnjati?,,,,izvꜵljn,jati,0,NaN
5731,20417,kvalifikovati,NaN,v.tr. ipf.,qualify,I,verb,?,?,kvalifikovati,kvalifikovati,kvalifik,kvalifikovati,|kvalifikacija,,,,,0,NaN
8064,17948,nezadovaljati,NaN,v.tr. ipf.,"displease, dissatisfy",NaN,verb,?,?,valjati,nezadovaljati,nezadoval,valjati,,,,,,0,NaN
10563,522,pohvaliti,NaN,v.tr. pf.,praise,NaN,verb,?,?,hvaliti,pohvaliti,pohval,hvaliti,,,,,,0,NaN
11118,11124,pozvaljati sobě,NaN,v.ipf.,afford,NaN,verb,?,?,pozvoliti,pozvaljati sobě,pozvaljati sobě,?zvaljati sobě?,,,,pozvꜵl,jati,0,NaN
11119,3156,pozvaljati,NaN,v.tr. ipf.,"allow, let, permit",NaN,verb,?,?,pozvoliti,pozvaljati,pozval,?zvaljati?,,,,pozvꜵl,jati,0,NaN
12968,30575,råzvaliti,NaN,v.refl. pf.,sprawl out,NaN,verb,råz’,valiti sę,råzvaliti,råzvaliti sę,råzval,valiti,,,,råzval,iti,0,NaN


In [448]:
"råzkrajati"[-7:-3]

'kraj'

In [445]:
verb_prefixes.query("base_verb == 'klåti'")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular,bas_verb
4926,26528,klåti,(kolje),v.tr. ipf.,"split, cleave",NaN,verb,?,?,klåti,klåti,klåti,klåti,,,,,,0,NaN
4927,32009,klåti,(kolje),v.tr. ipf.,slaughter,NaN,verb,?,?,klåti,klåti,klåti,klåti,,,,,,0,NaN
4928,36765,klåti,(kolje),v.tr. ipf.,"stab, sting, thrust, prick",NaN,verb,?,?,klåti,klåti,klåti,klåti,,,,,,0,NaN
12605,30221,råzkalati,NaN,v.tr. ipf.,"split, cleave",NaN,verb,råz’,kalati,klåti,råzkalati,råzkal,?kalati?,,,,,,0,NaN
12615,30233,råzklåti,(råzkolje),v.tr. pf.,"split, cleave",NaN,verb,råz’,klåti,klåti,råzklåti,råzklåti,klåti,,,,,,0,NaN
15664,32011,uklåti,(ukolje),v.tr. pf.,"sting, prick",NaN,verb,?,?,klåti,uklåti,uklåti,klåti,,,,,,0,NaN
17225,32007,zaklåti,(zakolje),v.tr. pf.,"stab, thrust, slaughter",NaN,verb,?,?,klåti,zaklåti,zaklåti,klåti,,,,,,0,NaN


In [423]:
verb_prefixes.query("isv.str.contains('čuvstvo')")



,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
14056,34940,sȯčuvstvovati,NaN,v.intr. ipf.,"sympathize, commiserate, feel compassion",NaN,verb,sȯ’,čuvstvovati,čuvati,sȯčuvstvovati,sȯčuvstv,?čuvstvovati?,,,,,,0


In [401]:
LSC = ('br', 'izbir', 'izbr', 'nabir', 'nabr', 'odbr', 'odbr', 'podbr', 'råzbir', 'råzbir', 'råzbr', 'råzbr', 'sbir', 'sbir', 'sȯbr', 'sȯbr', 'zabir', 'zabr')
LSC2 = [l.replace('br', 'bⁱr').replace("bir", 'bⁱr') for l in LSC]

common_suffix(LSC2)

'bⁱr'

In [404]:
Counter([bite_all_prefixes_off(l, "")[0] for l in LSC])

Counter({'br': 11, 'bir': 7})

In [397]:
stray_vowels = {"i": 'ⁱ', "e": 'ᵉ'}

stray_vowels

In [391]:
true_stem

'brån'

In [392]:
LSC

['brån', 'obrån', 'obrån', 'vȯzbrån', 'vȯzbrån', 'zabrån', 'zabrån']

In [393]:
RSC

('i+ti+', 'i+ti+', 'j+a+ti+', 'i+ti+', 'j+a+ti+', 'i+ti+', 'j+a+ti+')

In [407]:
verb_prefixes.query("base_verb == 'dějati' and isv.str.len() > 10")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular
12189,5479,protivdějati,NaN,v.intr. ipf.,counteract,NaN,verb,?,?,dějati,protivdějati,protivdě,?tivdějati?,,,,,,0
17046,13989,vzajemodějati,(vzajemoděje),v.intr. ipf.,interact,NaN,verb,?,?,dějati,vzajemodějati,vzajemodě,?jemodějati?,,,,,,0
17048,9090,vzajemodějstvovati,NaN,v.intr. ipf.,interact,NaN,verb,?,?,dějati,vzajemodějstvovati,vzajemodějstv,?jemodějstvovati?,,,,,,0


In [421]:
IDX = verb_prefixes.query("base_verb == 'žiti' and (en.str.contains('strip') or en.str.contains('expose'))").index

verb_prefixes.loc[IDX, "base_verb"] = "†nažiti"

In [417]:
IDX = verb_prefixes.query("base_verb == 'loviti' and isv.str.contains('zsl')").index

verb_prefixes.loc[IDX, 'base_verb'] = 'sloviti'

In [422]:
IDX = verb_prefixes.query("base_verb == 'niti' and isv.str.contains('ti')").index

verb_prefixes.loc[IDX, 'base_verb'] = ['niti', 'niti', 'sȯniti', 'podobniti', 'podobniti', 'podobniti', 'podobniti']
verb_prefixes.loc[14027, "_is_irregular"] = 1


In [454]:
for st in "uvoljniti valjati pozvoliti dozvoliti izvoljniti".split(" "):
    IDX = verb_prefixes.query("base_verb == @st").index
    verb_prefixes.loc[IDX, 'base_verb'] = 'voliti'


In [370]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
1246,35085,byti dȯlžen,NaN,v.aux. ipf.,"must, have to, ought to, should, be obliged to...",NaN,verb,?,?,byti,byti dȯlžen,byti dȯlžen,?byti dȯlžen?,,,,,
1248,10343,byti ostråžny,NaN,v.ipf.,"look out, watch out",NaN,verb,?,?,byti,byti ostråžny,byti ostråžny,?byti ostråžny?,,,,,
1249,17182,byti podobny,NaN,v.ipf.,"look alike, resemble",NaN,verb,?,?,byti,byti podobny,byti podobny,?byti podobny?,,,,,
1250,23728,byti prěhlåđeny,NaN,v.ipf.,have a cold,NaN,verb,?,?,byti,byti prěhlåđeny,byti prěhlåđeny,?byti prěhlåđeny?,,,,,
1251,2449,byti prinuđeny,NaN,v.ipf.,"have to, be forced to",NaN,verb,?,?,byti,byti prinuđeny,byti prinuđeny,?byti prinuđeny?,,,,,
1252,4190,byti sȯglåsny,NaN,v.ipf.,agree,NaN,verb,?,?,byti,byti sȯglåsny,byti sȯglåsny,?byti sȯglåsny?,,,,,
1253,593,byti,"(jesm, jesi, jest, jesmo, jeste, sųt; byl; bųdų)",v.intr. ipf.,be,NaN,verb,?,?,byti,byti,byti,byti,,,,,
1254,24896,byti,"(jesm, jesi, jest, jesmo, jeste, sųt; byl; bųdų)",v.intr. ipf.,"exist, be there, be available",NaN,verb,?,?,byti,byti,byti,byti,,,,,
2013,35371,dobyti ponovno,(dobųde),v.tr. pf.,regain,NaN,verb,?,?,byti,dobyti ponovno,dobyti ponovno,?byti ponovno?,,,,,
2014,3192,dobyti,(dobųde),v.tr. pf.,"get, obtain",NaN,verb,?,?,byti,dobyti,dobyti,byti,,,,,


In [375]:
signatures_cnt.most_common()

[('n', 68),
 ('l', 47),
 ('r', 44),
 ('č', 40),
 ('k', 36),
 ('p', 34),
 ('đ_d', 33),
 ('v', 30),
 ('g', 29),
 ('t_ć', 26),
 ('d', 25),
 ('š', 24),
 ('b', 21),
 ('ž', 19),
 ('z', 16),
 ('t', 14),
 ('š_s', 12),
 ('m', 10),
 ('s', 10),
 ('h', 8),
 ('z_ž', 6),
 ('d_v', 3),
 ('j_v', 2),
 ('č_n', 2),
 ('k_č', 2),
 ('đ', 2),
 ('ć', 2),
 ('t_s', 2),
 ('t_v_y', 1),
 ('đ_h', 1)]

In [360]:
verb_prefixes.isv.str[:-2]

5           abdikova
16           abonova
28         absorbova
29        abstrahova
43          adaptova
            ...     
17905           žuža
17913      råzpečęta
17914    råzpečętyva
17917           smog
17921     zapečętyva
Name: isv, Length: 4985, dtype: object

In [355]:
vc = verb_prefixes._prefix.value_counts()



In [395]:
verb_prefixes.loc[[9507, 12747, 12748], "base_verb"] = "oriti"
verb_prefixes.loc[[9644, 9645, 14236], "base_verb"] = "sporiti"
verb_prefixes.loc[[10444], "base_verb"] = "poriti"


In [396]:
# sporiti oriti poriti
verb_prefixes.query(' base_verb == "riti"')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix,_is_irregular


In [356]:
WTF = vc[vc == 1].index

In [357]:
verb_prefixes.query("_prefix in @WTF")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
2057,5345,dojdti,(dojde; došel),v.intr. pf.,arrive,NaN,verb,?,?,idti,dojdti,dojdti,?jdti?,,,doj,dti,
2185,7213,doråzuměti,NaN,v.refl. pf.,communicate,NaN,verb,?,?,uměti,doråzuměti sę,doråzum,uměti,,,dora,zum,ěti
4127,26207,iznemogti,NaN,v.intr. pf.,collapse,NaN,verb,?,?,mogti,iznemogti,iznemogti,mogti,,,izne,mꜵg,ti
4148,5354,izȯjdti,(izȯjde; izšel),v.intr. pf.,"exit, go out, quit",NaN,verb,?,?,idti,izȯjdti,izȯjdti,?ȯjdti?,,,izȯj,dti,
5608,4514,krvaviti,NaN,v.intr. ipf.,bleed,NaN,verb,?,?,krvaviti,krvaviti,krvav,krvaviti,,,krva,vi,ti
7187,781,najdti,(najde; našel),v.tr. pf.,find,NaN,verb,?,?,idti,najdti,najdti,?dti?,,,naj,dti,
7467,28206,nasmoliti,NaN,v.tr. pf.,tar,NaN,verb,?,?,moliti,nasmoliti,nasmol,moliti,,,nas,mol,iti
7668,20535,nedoråzuměti,NaN,v.tr. pf.,misunderstand,NaN,verb,?,?,uměti,nedoråzuměti,nedoråzum,uměti,,,nedora,zum,ěti
8302,19655,obezpokojiti,NaN,v.tr. pf.,"disturb, trouble, upset, bother",NaN,verb,obez’,pokojiti,†pokojiti,obezpokojiti,obezpoko,bezpokojiti,,,obez,poko,jiti
8892,181,odȯjdti,(odȯjde; odšel),v.intr. pf.,"go away, walk away, leave, backtrack",NaN,verb,?,?,idti,odȯjdti,odȯjdti,?ȯjdti?,,,odȯj,dti,


In [ ]:
#fix_LSC
# or get two LSC

In [ ]:
FUKKEN_IRREGULARS = 

  jestvovati
råzkalati
  odsųtstvovati
  prisųtstvovati 
  sųtstvovati
  provokovati
zaklę', 'zaklin



In [ ]:
IDX = verb_prefixes.query('isv.str.contains("oriti")').index

# verb_prefixes.loc[IDX, "base_verb"] = "prisposobiti"

In [380]:
verb_prefixes.loc[12233, "base_verb"] = "provokovati"

In [343]:
LSC

('prostir',
 'prostir',
 'prostr',
 'prostr',
 'råzprostir',
 'råzprostir',
 'råzprostr')

In [349]:

verb_prefixes.query('_stem == "" and base_verb == "krųtiti"')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
5601,19823,krųtiti,NaN,v.refl. ipf.,spin,NaN,verb,?,?,krųtiti,krųtiti sę,krųt,krųtiti,,,,,
5602,19820,krųtiti,NaN,v.tr. ipf.,"twist, twirl, whirl",NaN,verb,?,?,krųtiti,krųtiti,krųt,krųtiti,,,,,
12632,33855,råzkrųćati sę,NaN,v.tr. ipf.,"get going, gather speed",NaN,verb,råz’,krųćati sę,krųtiti,råzkrųćati sę,råzkrųć,?krųćati sę?,,,,,
12633,33853,råzkrųćati,NaN,v.tr. ipf.,"uncoil, untwist",NaN,verb,råz’,krųćati,krųtiti,råzkrųćati,råzkrųć,?krųćati?,,,,,
12634,33852,råzkrųtiti sę,NaN,v.tr. pf.,"get going, gather speed",NaN,verb,råz’,krųtiti sę,krųtiti,råzkrųtiti sę,råzkrųt,?krųtiti sę?,,,,,
12635,30254,råzkrųtiti,NaN,v.tr. pf.,"uncoil, untwist",NaN,verb,råz’,krųtiti,krųtiti,råzkrųtiti,råzkrųt,krųtiti,,,,,


In [339]:
verb_prefixes.query('_stem == "" and base_verb == "biti"')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
867,4049,biti,(bije),v.tr. ipf.,"hit, beat",NaN,verb,?,?,biti,biti,b,biti,,,,,
3921,23862,izbiti,(izbije),v.tr. pf.,"strike out, knock out",NaN,verb,?,?,biti,izbiti,izb,biti,|izbica,,,,
3922,23864,izbivati,NaN,v.tr. ipf.,"strike out, knock out",NaN,verb,?,?,biti,izbivati,izb,?bivati?,|izbica,,,,
7055,20513,nabiti,(nabije),v.tr. pf.,"stuff, fill, load (gun), tamp",NaN,verb,?,?,biti,nabiti,nab,biti,,,,,
7056,20511,nabivati,NaN,v.tr. ipf.,"stuff, fill, load (gun), tamp",NaN,verb,?,?,biti,nabivati,nab,?bivati?,,,,,
8712,29050,odbiti,(odbije),v.refl. pf.,be reflected,NaN,verb,?,?,biti,odbiti sę,odb,biti,,,,,
8713,34100,odbiti,(odbije),v.refl. pf.,have an effect,NaN,verb,?,?,biti,odbiti sę,odb,biti,,,,,
8714,20572,odbiti,(odbije),v.tr. pf.,"beat off, hit back (ball)",NaN,verb,?,?,biti,odbiti,odb,biti,,,,,
8715,20573,odbiti,(odbije),v.tr. pf.,repel,NaN,verb,?,?,biti,odbiti,odb,biti,,,,,
8716,34101,odbiti,(odbije),v.tr. pf.,reflect,NaN,verb,?,?,biti,odbiti,odb,biti,,,,,


In [342]:
IDX = verb_prefixes.query('isv.str.contains("spos")').index

verb_prefixes.loc[IDX, "base_verb"] = "prisposobiti"

In [348]:
verb_prefixes.to_pickle("verb_prefixes2.pkl")

In [ ]:
prisposobiti


In [325]:
RSC

('iti+',
 'յ+ati+',
 'յ+ati+',
 'iti+',
 'յ+ati+',
 'iti+',
 'iti+',
 'յ+ati+',
 'յ+ati+')

In [326]:
LSC

('povys',
 'povys',
 'prědvys',
 'prěvys',
 'prěvys',
 'vȯzvys',
 'vȯzvys',
 'vȯzvys',
 'vȯzvys')

In [322]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
1836,16731,děti,(děje/děne),v.refl. ipf.,"happen, take place",NaN,verb,?,?,děti,děti sę,d,děti,,,,,
1837,1191,děti,(děje/děne),v.tr. ipf.,"put, place",NaN,verb,?,?,děti,děti,d,děti,,,,,
7101,17837,naděti,NaN,v.tr. pf.,stuff (meat etc.),NaN,verb,?,?,děti,naděti,nad,děti,,,,,
7102,17838,naděti,NaN,v.tr. pf.,"don, put on",NaN,verb,?,?,děti,naděti,nad,děti,,,,,
7103,17840,naděvati,NaN,v.tr. ipf.,stuff (meat etc.),NaN,verb,?,?,děti,naděvati,naděv,?ěvati?,,,,,
7104,17841,naděvati,NaN,v.tr. ipf.,"don, put on",NaN,verb,?,?,děti,naděvati,naděv,?ěvati?,,,,,
8768,16840,oděti,(oděne),v.tr. pf.,"clothe, dress",NaN,verb,?,?,děti,oděti,od,děti,,,,,
10355,18249,poděti,NaN,v.refl. pf.,"disappear, be hidden",NaN,verb,?,?,děti,poděti sę,pod,děti,,,,,
11470,35288,prěoděti,NaN,v.refl. pf.,"dress up, change clothes",NaN,verb,?,?,děti,prěoděti sę,prěod,děti,,,,,
12510,33755,råzděti,NaN,v.refl. pf.,get undressed,NaN,verb,råz’,děti sę,děti,råzděti sę,råzd,děti,,,,,


In [321]:
LSC

('d',
 'd',
 'nad',
 'nad',
 'nad',
 'nad',
 'od',
 'pod',
 'prěod',
 'råzd',
 'råzd',
 'råzd',
 'råzd')

In [330]:
possible_suffixes

['yvati', 'ivati', 'nųti', 'jstv', 'iti', 'ěti', 'ati', 'ov', 'ěv', 'j', 'յ']

In [334]:
bite_all_suffixes_off(
    "doživeti", "", 
    ['yva', 'iva', 'ti', 'nų', 've', 'ov', 'ěv', 'ě', 'a', 'j', 'յ']
)

('doži', 've+ti+')

In [305]:
if last_vowels == {'a', 'o'}:
    print(1)
    # LSC = [AO_RE.sub(f"{AO}\\1", l) for l in LSC]


1


In [308]:
[AO_RE.sub(f"{AO}\\1", l) for l in LSC]

['dvꜵ', 'podvꜵ', 'råzdvꜵ', 'råzdvꜵ', 'udvꜵ']

In [304]:
last_vowels

{'a', 'o'}

In [303]:
LSC

['dvo', 'podvo', 'råzdva', 'råzdvo', 'udvo']

In [301]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
2453,25443,dvojiti,NaN,v.tr. ipf.,"double, duplicate",NaN,verb,?,?,dvojiti,dvojiti,dvo,dvojiti,,,,,
10503,17005,podvojiti,NaN,v.tr. ipf.,double,NaN,verb,?,?,dvojiti,podvojiti,podvo,dvojiti,,,,,
12534,18592,råzdvajati,NaN,v.tr. ipf.,divide in two,NaN,verb,råz’,dvajati,dvojiti,råzdvajati,råzdva,?dvajati?,,,,,
12536,18593,råzdvojiti,NaN,v.tr. pf.,divide in two,NaN,verb,råz’,dvojiti,dvojiti,råzdvojiti,råzdvo,dvojiti,,,,,
15631,13202,udvojiti,NaN,v.tr. pf.,"double, redouble, duplicate, geminate",NaN,verb,?,?,dvojiti,udvojiti,udvo,dvojiti,,,,,


In [296]:
common_suffix(LSC)

''

In [297]:
LSC

('dreti',
 'obdir',
 'obdreti',
 'oddir',
 'oddreti',
 'prodir',
 'prodreti',
 'råzdir',
 'råzdreti',
 'sdir',
 'sdreti')

In [292]:
RSC

('+',
 '+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 '+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ov+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'ati+',
 'iti+',
 'ati+')

In [289]:
g.isv.apply(lambda x: split_carefully(insert_YI(x, last_cons), true_stem))

{'m', 'ž', 'd', 't', 'v'}


ValueError: empty separator

In [288]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
1700,6979,davati råbotų,NaN,v.ipf.,"hire, employ",NaN,verb,?,?,davati,davati råbotų,davati råbotų,?davati råbotų?,,,,,
1701,22986,davati zajem,NaN,v.ipf.,lend,NaN,verb,?,?,davati,davati zajem,davati zajem,?davati zajem?,,,,,
1702,2898,davati,NaN,v.tr. ipf.,give,NaN,verb,?,?,davati,davati,dav,davati,|davec|davač,,,,
2036,1704,dodavati,NaN,v.tr. ipf.,add,NaN,verb,?,?,davati,dodavati,dodav,davati,,,,,
2037,7133,dodavati,NaN,v.tr. ipf.,upload,NaN,verb,?,?,davati,dodavati,dodav,davati,,,,,
3964,4779,izdavati,NaN,v.refl. ipf.,seem,NaN,verb,?,?,davati,izdavati sę,izdav,davati,,,,,
3965,34252,izdavati zamųž,NaN,v.tr. ipf.,marry off (to a man),NaN,verb,?,?,davati,izdavati zamųž,izdavati zamųž,?davati zamųž?,,,,,
3966,4853,izdavati,NaN,v.tr. ipf.,"publish, issue",NaN,verb,?,?,davati,izdavati,izdav,davati,,,,,
3967,17517,izdavati,NaN,v.tr. ipf.,give out,NaN,verb,?,?,davati,izdavati,izdav,davati,,,,,
7096,16974,nadavati,NaN,v.tr. ipf.,"bestow, confer, give",NaN,verb,?,?,davati,nadavati,nadav,davati,,,,,


In [ ]:
(4985, 1859)


In [299]:
def split_carefully(x, true_stem):
    if " " in x:
        x = x.split(" ")[0]
    x = x.replace('å', 'a')
    true_stem = true_stem.replace('å', 'a')

    if AO in true_stem:
        if true_stem.replace(AO, "a") in x:
            return list(x.partition(true_stem.replace(AO, "a")))
    return x.partition(true_stem.replace(AO, "o"))



In [ ]:
def split_carefully(x, true_stem):
    if " " in x:
        x = x.split(" ")[0]
    x = x.replace('å', 'a')
    true_stem = true_stem.replace('å', 'a')

    if AO in true_stem:
        if true_stem.replace(AO, "a") in x:
            return list(x.partition(true_stem.replace(AO, "a")))
    return x.partition(true_stem.replace(AO, "o"))



In [260]:
""[-1:]

''

In [253]:
LSC

('bd', 'snabd', 'snabděv')

In [219]:

    LSC, RSC = zip(
        *g.isv.apply(lambda x:
            bite_all_suffixes_off(insert_YI(x.replace("?", "")), "")
       )
    )
    
    #     if "†" in k:
    #         LSC += [
    #             bite_all_suffixes_off(
    #                 k.replace("†", ""), 
    #                 "")[0]
    #         ]

    # true_stem = list({s for s in LSC if all(rsc.startswith(s) for rsc in RSC)})
    true_stem = common_suffix(LSC)


{'a', 'o'}

['dogovꜵr',
 'dogovꜵr',
 'dogovꜵr',
 'dogovꜵr',
 'govꜵr',
 'izgovꜵr',
 'izgovꜵr',
 'nagovꜵr',
 'obgovꜵr',
 'obgovꜵr',
 'odgovꜵr',
 'odgovꜵr',
 'ogovꜵr',
 'ogovꜵr',
 'ogovꜵr',
 'ogovꜵr',
 'podgovꜵr',
 'pogovꜵr',
 'råzgovꜵr',
 'råzgovꜵr']

In [221]:
RSC

('j+ati+',
 'j+ati+',
 'iti+',
 'iti+',
 'iti+',
 'j+ati+',
 'iti+',
 'iti+',
 'j+ati+',
 'iti+',
 'iti+',
 'iti+',
 'j+ati+',
 'j+ati+',
 'iti+',
 'iti+',
 'iti+',
 'iti+',
 'j+ati+',
 'iti+')

In [194]:
vc = verb_prefixes._prefix.value_counts()

vc[vc > 1]

          3692
råz        100
iz          99
o           87
za          72
          ... 
ne           2
upodob       2
izob         2
upodab       2
privo        2
Name: _prefix, Length: 100, dtype: int64

In [203]:
verb_prefixes._stem.apply(len).value_counts()


0     2894
3      723
1      411
4      377
6      186
5      172
7       85
2       83
8       28
9       14
10       8
11       4
Name: _stem, dtype: int64

In [202]:
vc[vc == 2].index

Index(['izsy', 'shrå', 'osrå', 'os', 'hlå', 'ro', 'sbi', 'hrå', 'ogra',
       'osvoba', 'ogovo', 'ogova', 'sna', 'sȯb', 'odpro', 'zadovo', 'odgovo',
       'osu', 'go', 'srå', 'råzprostra', 'doga', 'prěråz', 'prěo', 'prěob',
       'prohlå', 'prěds', 'prědpo', 'prone', 'dogova', 'dogovo', 'podråz',
       'bez', 'råzo', 'råzpo', 'råzprostrå', 'odb', 'odsy', 'navo', 'umo',
       'izvo', 'izsu', 'mų', 'izrå', 'izpi', 'naslå', 'nasla', 'izp', 'urå',
       'ne', 'upodob', 'izob', 'upodab', 'privo'],
      dtype='object')

In [ ]:
#govoriti
#gnati

In [214]:
g = verb_prefixes.query("base_verb == 'govoriti'")

In [218]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
2047,5534,dogovarjati,NaN,v.refl. ipf.,agree,NaN,verb,?,?,govoriti,dogovarjati sę,dogovar,?govarjati?,,,dogova,r,jati
2048,2899,dogovarjati,NaN,v.tr. ipf.,"negotiate, contract, settle, agree upon",NaN,verb,?,?,govoriti,dogovarjati,dogovar,?govarjati?,,,dogova,r,jati
2051,4192,dogovoriti,NaN,v.refl. pf.,agree,NaN,verb,?,?,govoriti,dogovoriti sę,dogovor,govoriti,,,dogovo,r,iti
2052,655,dogovoriti,NaN,v.tr. pf.,"negotiate, contract, settle, agree upon",NaN,verb,?,?,govoriti,dogovoriti,dogovor,govoriti,,,dogovo,r,iti
3189,754,govoriti,NaN,v.tr. ipf.,"say, speak, talk",NaN,verb,?,?,govoriti,govoriti,govor,govoriti,,govor,govo,r,iti
3992,5553,izgovarjati,NaN,v.tr. ipf.,pronounce,NaN,verb,?,?,govoriti,izgovarjati,izgovar,?govarjati?,,,izgova,r,jati
3994,4370,izgovoriti,NaN,v.tr. pf.,pronounce,NaN,verb,?,?,govoriti,izgovoriti,izgovor,govoriti,|izgovorka,,izgovo,r,iti
7154,27984,nagovoriti,NaN,v.tr. ipf.,talk a lot,NaN,verb,?,?,govoriti,nagovoriti,nagovor,govoriti,,,nagovo,r,iti
8310,9783,obgovarjati,NaN,v.tr. ipf.,"discuss, debate, talk over",NaN,verb,?,?,govoriti,obgovarjati,obgovar,?govarjati?,,,obgova,r,jati
8311,9782,obgovoriti,NaN,v.tr. pf.,"discuss, debate, talk over",NaN,verb,?,?,govoriti,obgovoriti,obgovor,govoriti,,,obgovo,r,iti


In [217]:
g._suffix

2047       jati
2048       jati
2051        iti
2052        iti
3189        iti
3992       jati
3994        iti
7154        iti
8310       jati
8311        iti
8780        iti
8781        iti
9170       jati
9171       jati
9173        iti
9174        iti
10363       iti
10539       iti
12555    åzgova
12557    åzgovo
Name: _suffix, dtype: object

In [215]:
g.isv.str.replace("gn", "gᵃn").apply(get_vowels)

2047     ooaai
2048     ooaai
2051     oooii
2052     oooii
3189      ooii
3992     ioaai
3994     iooii
7154     aooii
8310     ooaai
8311     oooii
8780     oooii
8781     oooii
9170     ooaai
9171     ooaai
9173     oooii
9174     oooii
10363    oooii
10539    oooii
12555    åoaai
12557    åooii
Name: isv, dtype: object

In [213]:
g.isv.apply(get_vowels)

2040     oaai
2044      oai
3057       ai
3974     iaai
3986      iai
7138     aaai
7146      aai
12006    oaai
12539    åaai
12550     åai
Name: isv, dtype: object

In [210]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
2040,25287,doganjati,NaN,v.tr. ipf.,catch up with,NaN,verb,?,?,gnati,doganjati,dogan,?ganjati?,,,doga,n,jati
2044,25286,dognati,(dogone),v.tr. pf.,catch up with,NaN,verb,?,?,gnati,dognati,dogn,gnati,,,dog,n,ati
3057,1815,gnati,(gone),v.tr. ipf.,"drive, chase",NaN,verb,?,?,gnati,gnati,gn,gnati,,,g,n,ati
3974,1692,izganjati,NaN,v.tr. ipf.,"expel, banish, oust, exile",NaN,verb,?,?,gnati,izganjati,izgan,?ganjati?,,,izga,n,jati
3986,1695,izgnati,(izgone),v.tr. pf.,"expel, banish, oust, exile",NaN,verb,?,?,gnati,izgnati,izgn,gnati,,,izg,n,ati
7138,32811,naganjati,NaN,v.tr. ipf.,"prod, shoo",NaN,verb,?,?,gnati,naganjati,nagan,?ganjati?,,,,n,aga
7146,27975,nagnati,(nagone),v.tr. pf.,"prod, shoo",NaN,verb,?,?,gnati,nagnati,nagn,gnati,,,,n,ag
12006,14942,proganjati,NaN,v.tr. ipf.,persecute,NaN,verb,?,?,gnati,proganjati,progan,?ganjati?,,,proga,n,jati
12539,30168,råzganjati,NaN,v.tr. ipf.,"disperse, dispel",NaN,verb,råz’,ganjati,gnati,råzganjati,råzgan,?ganjati?,,,råzga,n,jati
12550,30176,råzgnati,(råzgone),v.tr. pf.,"disperse, dispel",NaN,verb,råz’,gnati,gnati,råzgnati,råzgn,gnati,,,råzg,n,ati


In [204]:
verb_prefixes.query("_prefix.str.endswith('a')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
2040,25287,doganjati,NaN,v.tr. ipf.,catch up with,NaN,verb,?,?,gnati,doganjati,dogan,?ganjati?,,,doga,n,jati
2041,25285,dogarjati,NaN,v.tr. ipf.,burn down,NaN,verb,?,?,gorěti,dogarjati,dogar,?garjati?,,,doga,r,jati
2047,5534,dogovarjati,NaN,v.refl. ipf.,agree,NaN,verb,?,?,govoriti,dogovarjati sę,dogovar,?govarjati?,,,dogova,r,jati
2048,2899,dogovarjati,NaN,v.tr. ipf.,"negotiate, contract, settle, agree upon",NaN,verb,?,?,govoriti,dogovarjati,dogovar,?govarjati?,,,dogova,r,jati
2294,435,dozvaljati,NaN,v.tr. ipf.,"let, permit, allow",NaN,verb,?,?,dozvoliti,dozvaljati,dozval,?zvaljati?,,,dozva,l,jati
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17500,23255,zašiti,(zašije),v.tr. pf.,"sew, stitch",NaN,verb,?,?,šiti,zašiti,zaš,šiti,,,za,š,iti
17518,14186,zatrimati,NaN,v.tr. pf.,detain,NaN,verb,?,?,trimati,zatrimati,zatrim,trimati,,,za,trim,ati
17519,14185,zatrimyvati,NaN,v.tr. ipf.,detain,NaN,verb,?,?,trimati,zatrimyvati,zatrim,?trimyvati?,,,za,trim,yvati
17521,5204,zatvarjati,NaN,v.tr. ipf.,"close, shut",NaN,verb,?,?,tvoriti,zatvarjati,zatvar,?tvarjati?,,,zatva,r,jati


In [189]:
verb_prefixes[(verb_prefixes.prefix == '?')]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
5,11,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija,,,,
16,6119,abonovati,NaN,v.tr. ipf.,"subscribe, engage",F,verb,?,?,abonovati,abonovati,abon,abonovati,,,,,
28,19625,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija,,,,
29,24056,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,,,,
43,24070,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17894,1490,žrtvovati,NaN,v.tr. ipf.,sacrifice,NaN,verb,?,?,žrtvovati,žrtvovati,žrtv,žrtvovati,,žrtva,,,
17904,2627,žuvati,(žuje),v.intr. ipf.,"chew, masticate",NaN,verb,?,?,žuvati,žuvati,žuv,žuvati,,,,,
17905,19473,žužati,NaN,v.intr. ipf.,buzz,NaN,verb,?,?,žužati,žužati,žuž,žužati,,,,,
17917,-30731,smogti,NaN,v.tr. pf.,"manage to, be able to, cope, make it",NaN,verb,?,?,mogti,smogti,smogti,mogti,,,,,


In [176]:
common_suffix(LSC)

In [184]:
LSC

('povys',
 'povys',
 'prědvys',
 'prěvys',
 'prěvys',
 'vȯzvys',
 'vȯzvys',
 'vȯzvys',
 'vȯzvys')

In [185]:
RSC

('iti+',
 'յ+ati+',
 'յ+ati+',
 'iti+',
 'յ+ati+',
 'iti+',
 'iti+',
 'յ+ati+',
 'յ+ati+')

In [172]:
for i, row in g.isv.apply(lambda x: insert_YI(x).split(true_stem)).items():
    print(row)


['', 'iti']
['za', 'iti']
['za', 'iti']
['za', 'յati']
['za', 'յati']


In [169]:
g.isv.apply(lambda x: x.split(true_stem))


968          [, iti]
17085      [za, iti]
17086      [za, iti]
17088    [zablųđati]
17089    [zablųđati]
Name: isv, dtype: object

In [126]:
true_stem = list({s for s in LSC if all(rsc.startswith(s) for rsc in RSC)})
true_stem

[]

In [130]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
11075,3687,povysiti,NaN,v.tr. pf.,"increase, heighten",NaN,verb,?,?,†vysiti,povysiti,povys,?iti?,,,,,
11076,5546,povyšati,NaN,v.tr. ipf.,"increase, heighten",NaN,verb,?,?,†vysiti,povyšati,povyš,?šati?,,,,,
11371,23357,prědvyšati,NaN,v.tr. ipf.,"prefer, like better",NaN,verb,?,?,†vysiti,prědvyšati,prědvyš,?šati?,,,,,
11615,22729,prěvysiti,NaN,v.tr. pf.,"surpass, exceed",NaN,verb,?,?,†vysiti,prěvysiti,prěvys,?iti?,,,,,
11616,22728,prěvyšati,NaN,v.tr. ipf.,"surpass, exceed",NaN,verb,?,?,†vysiti,prěvyšati,prěvyš,?šati?,,,,,
16784,22737,vȯzvysiti,NaN,v.tr. pf.,"raise, elevate",NaN,verb,vȯz’,vysiti,†vysiti,vȯzvysiti,vȯzvys,?iti?,,,,,
16785,22742,vȯzvysiti,NaN,v.tr. pf.,"extol, exalt",NaN,verb,vȯz’,vysiti,†vysiti,vȯzvysiti,vȯzvys,?iti?,,,,,
16786,22736,vȯzvyšati,NaN,v.tr. ipf.,"raise, elevate",NaN,verb,vȯz’,vyšati,†vysiti,vȯzvyšati,vȯzvyš,?šati?,,,,,
16787,22739,vȯzvyšati,NaN,v.tr. ipf.,"extol, exalt",NaN,verb,vȯz’,vyšati,†vysiti,vȯzvyšati,vȯzvyš,?šati?,,,,,


In [129]:
[bite_all_prefixes_off(l, "vys") for l in LSC]

[('vys', 'po’'),
 ('vys', 'po’'),
 ('vys', 'prěd’'),
 ('vys', 'prě’'),
 ('vys', 'prě’'),
 ('vys', 'vȯz’'),
 ('vys', 'vȯz’'),
 ('vys', 'vȯz’'),
 ('vys', 'vȯz’')]

In [182]:
from itertools import zip_longest, takewhile

def all_equal(items: (tuple, list, str)) -> bool:
    '''
    A helper function to test if 
    all items in the given iterable 
    are identical. 

    Arguments:
    item -> the given iterable to be used

    eg.
    >>> all_same([1, 1, 1])
    True
    >>> all_same([1, 1, 2])
    False
    >>> all_same((1, 1, 1))
    True
    >> all_same((1, 1, 2))
    False
    >>> all_same("111")
    True
    >>> all_same("112")
    False
    '''
    return all(item == items[0] for item in items)


def common_suffix(strings: (list, tuple), _min: int=0, _max: int=100) -> str:
    '''
    Given a list or tuple of strings, find the common suffix
    among them. If a common suffix is not found, an empty string
    will be returned.

    Arguments:
    strings -> the string list or tuple to
    be used.

    _min, _max - > If a common suffix is  found, 
    Its length will be tested against the range _min 
    and _max. If its length is not in the range, and
    empty string will be returned, otherwise the suffix
    is returned 

    eg.
    >>> common_suffix([rhyme', 'time', 'mime'])
    'me'
    >>> common_suffix(('boo', 'foo', 'goo'))
    'oo'
    >>> common_suffix(['boo', 'foo', 'goz'])
    ''
    '''
    suffix = ""
    strings = [string[::-1] for string in strings]
    for tup in zip_longest(*strings):
        if all_equal(tup):
            suffix += tup[0]
        else:
            if _min <= len(suffix) <= _max:
                return suffix[::-1]
            else:
                return ''
    return suffix[::-1]

common_suffix(LSC)

'dokon'

In [179]:
common_suffix(['dok', 'dok'])

In [167]:
true_stem = common_suffix(LSC)
true_stem

'vys'

In [ ]:
[s for s in stem_cands if ]

In [ ]:

true_stem = list({s for s in LSC if all(rsc.startswith(s) for rsc in RSC)})


In [125]:
RSC

('iti+',
 'յ+ati+',
 'յ+ati+',
 'iti+',
 'յ+ati+',
 'iti+',
 'iti+',
 'յ+ati+',
 'յ+ati+')

In [131]:
LSC

('povys',
 'povys',
 'prědvys',
 'prěvys',
 'prěvys',
 'vȯzvys',
 'vȯzvys',
 'vȯzvys',
 'vȯzvys')

In [87]:
verb_prefixes.left_stem_cand.str[-1].value_counts()

d    445
i    402
r    387
v    385
t    382
n    377
l    337
k    266
č    221
z    210
p    181
ž    176
š    153
g    152
s    146
b    145
m    123
ć    103
h     85
đ     71
o     62
a     54
ě     31
ų     21
u     15
e     13
y     11
c     10
ŕ      8
ť      4
)      4
f      3
ę      1
Name: left_stem_cand, dtype: int64

In [64]:
true_stem



['gy']

In [93]:
import string



In [95]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [97]:
INT_ENDS = [x for x in verb_prefixes.isv.str[-4:].unique() if x.endswith("ti") and (x[-4] not in string.ascii_letters)]



In [112]:
verb_prefixes.query("isv.str.endswith('žiti')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
950,24702,bližiti,NaN,v.refl. ipf.,approach,NaN,verb,?,?,bližiti,bližiti sę,bliž,bližiti,,,,bliž,iti
2117,20364,doložiti,NaN,v.tr. pf.,add,NaN,verb,?,?,†ložiti,doložiti,dolož,?ložiti?,,,do,lož,iti
2122,25312,dȯlžiti,NaN,v.tr. ipf.,lengthen,NaN,verb,?,?,dȯlžiti,dȯlžiti,dȯlž,dȯlžiti,|dȯlžnik,,,dȯlž,iti
2305,4486,dožiti,(dožive),v.tr. pf.,live out,NaN,verb,?,?,žiti,dožiti,dož,žiti,,,,,
2370,7333,družiti,NaN,v.refl. ipf.,make friends,NaN,verb,?,?,družiti,družiti sę,druž,družiti,|družka|družica|družstvo,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17278,14112,založiti,NaN,v.tr. pf.,"pawn, hock",NaN,verb,?,?,†ložiti,založiti,založ,?ložiti?,|založnik,,za,lož,iti
17453,20319,zaslužiti,NaN,v.tr. pf.,deserve,NaN,verb,?,?,zaslužiti,zaslužiti,zasluž,služiti,,,,,
17540,23978,zauvažiti,NaN,v.tr. pf.,notice,NaN,verb,?,?,zauvažati,zauvažiti,zauvaž,važiti,,,,,
17845,28,žiti,(žive),v.intr. ipf.,live,NaN,verb,?,?,žiti,žiti,ž,žiti,,,,,


In [ ]:
gj ž (begati)
kj č
cj č
dj đ
hj žđ


In [111]:
E = INT_ENDS[11]
print(E)
verb_prefixes.query("isv.str.endswith(@E)")

žati


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
2384,17344,dŕžati,#(dŕži),v.refl. ipf.,"hold onto something, cling to something",NaN,verb,?,?,dŕžati,dŕžati sę něčego,dŕž,dŕžati,|dŕžka,,,dŕž,ati
2385,2617,dŕžati,(dŕži),v.tr. ipf.,"hold, keep",NaN,verb,?,?,dŕžati,dŕžati,dŕž,dŕžati,|dŕžka,,,dŕž,ati
2386,25399,držati,(drži),v.intr. ipf.,"shudder, shiver, tremble",NaN,verb,?,?,držati,držati,drž,držati,,,,,
4145,1865,izobražati,NaN,v.tr. ipf.,"depict, portray",NaN,verb,?,?,raziti,izobražati,izobraž,?ražati?,,,,,
4230,36717,izražati,NaN,v.refl. ipf.,express oneself,NaN,verb,?,?,raziti,izražati sę,izraž,?ražati?,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17146,36514,zadŕžati,(zadŕži),v.tr. pf.,"pause, stop",NaN,verb,?,?,dŕžati,zadŕžati,zadŕž,dŕžati,|zadŕžnik,,za,dŕž,ati
17316,2342,zamražati,NaN,v.tr. ipf.,freeze,NaN,verb,?,?,mråziti,zamražati,zamraž,?mražati?,,,,,
17422,21244,zaražati,NaN,v.tr. ipf.,"infect, contaminate",NaN,verb,?,?,raziti,zaražati,zaraž,?ražati?,,,,,
17539,23977,zauvažati,NaN,v.tr. ipf.,notice,NaN,verb,?,?,zauvažati,zauvažati,zauvaž,uvažati,,,,,


In [ ]:
čati

In [122]:
[(w, insert_YI(w)) for w in g.isv.unique()]

[('povysiti', 'povysiti'),
 ('povyšati', 'povysյati'),
 ('prědvyšati', 'prědvysյati'),
 ('prěvysiti', 'prěvysiti'),
 ('prěvyšati', 'prěvysյati'),
 ('vȯzvysiti', 'vȯzvysiti'),
 ('vȯzvyšati', 'vȯzvysյati')]

In [114]:
LSC, RSC = zip(
    *g.isv.apply(lambda x:
        bite_all_suffixes_off(insert_YI(x), "")
   )
)
LSC, RSC

(('povys',
  'povys',
  'prěds',
  'prěvs',
  'prěvs',
  'vȯzvs',
  'vȯzvs',
  'vȯzvs',
  'vȯzvs'),
 ('iti+',
  'յ+ati+',
  'յ+ati+',
  'iti+',
  'յ+ati+',
  'iti+',
  'iti+',
  'յ+ati+',
  'յ+ati+'))

In [81]:
RSC = [rsc.replace("?", "") for rsc in g.right_stem_cand.values.tolist()]
LSC = g.left_stem_cand.values.tolist()

# LSC = g.left_stem_cand.str.replace("š$", "s" + YI).tolist()



In [83]:
...,*zip(LSC, RSC)

(Ellipsis,
 ('povys', 'iti'),
 ('povyš', 'šati'),
 ('prědvyš', 'šati'),
 ('prěvys', 'iti'),
 ('prěvyš', 'šati'),
 ('vȯzvys', 'iti'),
 ('vȯzvys', 'iti'),
 ('vȯzvyš', 'šati'),
 ('vȯzvyš', 'šati'))

In [77]:
RSC

['iti', 'šati', 'šati', 'iti', 'šati', 'iti', 'iti', 'šati', 'šati']

In [78]:
LSC

['povys',
 'povysյ',
 'prědvysյ',
 'prěvys',
 'prěvysյ',
 'vȯzvys',
 'vȯzvys',
 'vȯzvysյ',
 'vȯzvysյ']

In [80]:
true_stem = list({s for s in LSC if all(rsc.startswith(s) for rsc in RSC)})
true_stem

[]

In [63]:
bite_all_suffixes_off('gynųti', '')

('gy', 'nųti+')

C:\Users\79165\AppData\Local\Temp\ipykernel_9748\718354869.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  g.left_stem_cand.str.replace("š$", "s" + YI)


11075       povys
11076      povysյ
11371    prědvysյ
11615      prěvys
11616     prěvysյ
16784      vȯzvys
16785      vȯzvys
16786     vȯzvysյ
16787     vȯzvysյ
Name: left_stem_cand, dtype: object

In [71]:
LSC

['povys',
 'povyš',
 'prědvyš',
 'prěvys',
 'prěvyš',
 'vȯzvys',
 'vȯzvys',
 'vȯzvyš',
 'vȯzvyš',
 'vys']

In [72]:
RSC

['?iti?',
 '?šati?',
 '?šati?',
 '?iti?',
 '?šati?',
 '?iti?',
 '?iti?',
 '?šati?',
 '?šati?']

In [54]:
verb_prefixes._suffix.value_counts()

                 4197
ati               233
iti               200
jati               90
nųti               81
yvati              60
ěti                30
ovati              29
ivati              15
ovyvati            11
ěvati               9
oviti               5
jiti                5
avati               3
avjati              2
ati sȯlzy           1
ati dyh             1
nųti sȯlzy          1
yvati roljų         1
jivati              1
iti perami          1
nųti glåvojų        1
iti o               1
nųti těnj na        1
ati ponovno         1
ati glåvojų         1
oviti rybų          1
ati glådovkų        1
yvati ponovno       1
ivati dyh           1
Name: _suffix, dtype: int64

In [52]:
len(verb_prefixes), len(verb_prefixes.query('_stem != ""'))

(4985, 788)

In [49]:
g

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix
11075,3687,povysiti,NaN,v.tr. pf.,"increase, heighten",NaN,verb,?,?,†vysiti,povysiti,povys,?iti?,,,,,
11076,5546,povyšati,NaN,v.tr. ipf.,"increase, heighten",NaN,verb,?,?,†vysiti,povyšati,povyš,?šati?,,,,,
11371,23357,prědvyšati,NaN,v.tr. ipf.,"prefer, like better",NaN,verb,?,?,†vysiti,prědvyšati,prědvyš,?šati?,,,,,
11615,22729,prěvysiti,NaN,v.tr. pf.,"surpass, exceed",NaN,verb,?,?,†vysiti,prěvysiti,prěvys,?iti?,,,,,
11616,22728,prěvyšati,NaN,v.tr. ipf.,"surpass, exceed",NaN,verb,?,?,†vysiti,prěvyšati,prěvyš,?šati?,,,,,
16784,22737,vȯzvysiti,NaN,v.tr. pf.,"raise, elevate",NaN,verb,vȯz’,vysiti,†vysiti,vȯzvysiti,vȯzvys,?iti?,,,,,
16785,22742,vȯzvysiti,NaN,v.tr. pf.,"extol, exalt",NaN,verb,vȯz’,vysiti,†vysiti,vȯzvysiti,vȯzvys,?iti?,,,,,
16786,22736,vȯzvyšati,NaN,v.tr. ipf.,"raise, elevate",NaN,verb,vȯz’,vyšati,†vysiti,vȯzvyšati,vȯzvyš,?šati?,,,,,
16787,22739,vȯzvyšati,NaN,v.tr. ipf.,"extol, exalt",NaN,verb,vȯz’,vyšati,†vysiti,vȯzvyšati,vȯzvyš,?šati?,,,,,


In [37]:
verb_prefixes.columns = ['id', 'isv', 'addition', 'partOfSpeech', 'en', 'genesis', 'pos',
       'verb_stem', 'base_verb', 'isv_orig', 'left_stem_cand',
       'right_stem_cand', 'derived_nouns', 'base_noun', 'prefix', 'stem', 'suffix']

In [42]:
verb_prefixes.query('_stem == "bav"')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun,_prefix,_stem,_suffix


In [30]:
g.isv

654         baviti
1983      dobaviti
1984     dobavjati
3900      izbaviti
3901      izbaviti
3902      izbaviti
3903     izbavjati
3904     izbavjati
3905     izbavjati
17070     zabaviti
17071    zabavjati
Name: isv, dtype: object

In [ ]:
"ꜵ"

In [26]:
local_prefixes = g.isv.apply(lambda x: bite_all_prefixes_off(x, 'bav')[1])
local_prefixes

654      ’
1983     ’
1984     ’
3900     ’
3901     ’
3902     ’
3903     ’
3904     ’
3905     ’
17070    ’
17071    ’
Name: isv, dtype: object

In [20]:
bite_all_suffixes_off(k, "")

('bav', 'iti+')

In [27]:
"Ꜵ".lower()

'ꜵ'

In [15]:
g.partOfSpeech.value_counts()

v.tr. ipf.      5
v.tr. pf.       4
v.refl. pf.     1
v.refl. ipf.    1
Name: partOfSpeech, dtype: int64

In [5]:
for k, g in verb_prefixes.groupby("en"):
    if k[0] != "#" and len(g) > 1 and len(g.base_verb.unique()) > 1:
        print(k, g.base_verb.unique())
        # break


abandon ['' 'odstųpati' 'pustiti' 'pušćati']
abdicate ['abdikovati' 'kazati']
abduct, kidnap ['ti' 'nositi']
absorb ['absorbovati' 'piti']
abuse, misuse ['žiti' 'zloupotrěbiti']
accept, receive ['jęti' 'prijmati']
accumulate ['akumulovati' 'gromaditi']
acquaint ['znati' 'poznavati']
acquire ['byti' 'byvati']
acquit ['opravdati' 'uneviniti']
add ['baviti' 'dati' 'davati' '†kladati' '†ložiti']
add, attach ['dati' 'davati']
address, appeal to ['obraćati' '']
adjust ['staviti' 'stavjati' 'praviti']
admit, grant admittance ['pustiti' 'pušćati']
advance ['avansovati' 'idti']
advise, counsel ['sȯvětovati' '']
agree ['byti' 'govoriti']
aim ['cěliti' 'měriti']
allow, permit ['pustiti' 'pušćati']
amputate ['amputovati' 'imati' 'jęti']
anger, make angry ['gněvati' 'sŕditi']
announce ['obvěstiti' 'oznaniti']
announce, declare, herald ['glåsiti' 'oglašati']
answer, reply ['odgovarjati' 'govoriti' 'odpovědati']
appear (before a court) ['tati' 'stavati']
approach ['bližiti' 'hoditi' 'odȯjdti' '']
app

protrude ['sunųti' 'suvati']
provide, supply ['staviti' 'stavjati']
publish, issue ['dati' 'davati']
pull tight, draw tight, stretch out ['tęgati' 'tęgnųti']
pull, draw ['tęgati' 'tęgnųti']
pull, tear, tug, jerk ['trgati' 'trgnųti']
punish ['karati' 'kazniti']
put aside, set aside ['staviti' 'stavjati']
put away, put off, put aside, put back ['odkladati' 'odložiti']
put under ['odkladati' '†lagati' 'odložiti']
put, place ['děti' 'staviti' 'stavjati']
quarrel ['pirati' 'ti']
quote ['vesti' 'voditi' 'citovati']
raise ['ti' 'nositi']
rake ['grabiti' 'grebti']
reach crawling ['pȯlzati' 'pȯlzti']
rebel, revolt ['tati' 'stavati']
recognise, recognize ['znati' 'poznavati']
recommend ['rųčiti' 'rekomendovati']
redirect ['adresovati' 'měriti']
reduce ['redukovati' 'smenšati']
reflect ['biti' 'raziti']
regain ['byti' 'byvati' 'iskati']
register ['registrovati' 'pisati']
regret ['' 'žaliti']
relate to, feel about, concern, treat, behave toward ['ti' 'nositi']
release, unleash, launch (ship) ['pus